# Make patterns and calculate colocalizations

Purpose:
* Make particle sets (point patterns)
* Calculate colocalizations between these patterns
* Need to be executed before analysis.ipynb notebook

Usage:
* Execute all cells in Initializatin and General parameters ... sections
* Execute all cells in Make point ... section to make point patterns
* Execute all cells (in any order) in Calculate 2-coloc ... and Calculate 3-coloc ... sections to calculate colocalizations
* Takes 1-2 h to execute the entire notebook

Saves data:
* Particle patterns in particles/
* Colocalization results in coloc/

### Pattern and colocalization names

Particle sets A, B and C here correspond to sets P, Q and R in the manuscript.

Pattern names:
* ARandom[Full, 08, 06, 05]: Randomly distributed particle set A on the entire area, 80%, 60% an 50% of the entire area, respectively
* ARandom[ExclFull, Extend2]:  Randomly distributed particle set A with particle exclusion distance and on an extended area, respectively
* BrfCoclustAr[area_A]p[fraction_points_A][variant]-pkd-d[interaction_distance]: Set B that locally interacts with set A in 1-to-1 mode, where
  - area_A: (f, 08, 06, 05) Area over which A is distributed (full, 80%, 60% and 50%) respectively
  - fraction_points_A: (03, 02, 015, ...) Fraction of points in A that are involved in the interaction (03, 0.2, 0.15, ..., respectively)
  - variant (used only for 3-colocalizations): ('k', '') Common and independent interacting subset variants, respectively 
  - interaction_distance: (5, 10, 15) Interaction distance in pixels
* BrfCoclustAr[area_A]p[fraction_points_A]-p[fraction_points_B]-d[interaction_distance]: Set B that locally interacts with set A in multimeric mode (like 1-to-1 mode, except that fraction of interacting points is specified separately for the two sets)

Colocalization names:
* Composed of pattern names specified in the colocalization order and separated by '_'
* For example:
    - CRandom06_ARandomFull: 2-colocalization, univariate clustering
    - ARandomFull_BrfCoclustArfp01k-pkd-d10_CrfCoclustArfp01k-pkd-d10: 3-colocalization, local interactions 1-to-1 mode, common interacting subset variant

## Initialization

In [1]:
import os
import sys
import itertools

import numpy as np
import scipy as sp
import pandas as pd 

import pyto
from pyto.io.pandas_io import PandasIO
import pyto.spatial.point_pattern as point_pattern
import pyto.spatial.coloc_functions as col_func
import pyto.spatial.coloc_theory as coloc_theory
from pyto.spatial.particle_sets import ParticleSets
from pyto.spatial.multi_particle_sets import MultiParticleSets
from pyto.spatial.coloc_functions import plot_p, plot_data
from pyto.spatial.coloc_lite import ColocLite

# matplotlib 
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

# common
import common
from common import n_A, n_B, n_C
from common import rectangle_50_area, rectangle_05_area, rectangle_06_area
from common import rectangle_08_area, rectangle_full_area
from common import rectangle_05w_05e_over, rectangle_05sw_05ne_over
from common import rectangle_06w_06e_over, rectangle_06sw_06ne_over

# convenience
#import ipdb
%autosave 0
sys.version

Autosave disabled


'3.13.3 | packaged by conda-forge | (main, Apr 14 2025, 20:44:03) [GCC 13.3.0]'

In [2]:
# Settings

# force showing all rows in a table (default 60 rows)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)

# set hostname
hostname = os.uname()[1]
print(f"Host: {hostname}")

Host: rauna


## General parameters and definitions

In [3]:
# General params

# random number seed used to generate random patterns 
seed = 123456

# path to particles (patterns) table
mps_path = 'particles/mps.pkl'

# colocalization distance [nm]
distance = np.arange(1, 31)

# N simulations
n_simul = 20

# colocalization name mode
name_mode = '_'

In [4]:
# Patterns and regions parameters

# region (bit elongated 80 000 pix^2) and small rectangle positions
region_x = [100, 500]
region_y = [200, 400]
region_z = 20
small_x = [200, 250]
small_y = [300, 350]
extend_x = [0, 1600]
extend_y = [100, 150]
extend2_x = [0, 4000]
extend2_y = [100, 120]
big_x = [120, 480]
big_y = [210, 390]
big_06_x = [150, 450]
big_06_y = [220, 380]
big_05_x = [175, 425]
big_05_y = [220, 380]

# region images
region_id = 1
region_path = 'regions/flat_region.mrc'
region_extend_path = 'regions/flat_region_extend.mrc'
region_extend2_path = 'regions/flat_region_extend2.mrc'
#region_08_path = 'flat_region_big.mrc'

# pixel size in nm
pixel_size_nm = 1

# exclusion distance [pixels]
exclusion = 5

# Make region-related variables from parameters
# needed to make "experimental" and simulated particles
rectangle_50 = [[small_x[0], small_y[0], region_z], 
                [small_x[1], small_y[1], region_z+1]]
rectangle_full = [[region_x[0], region_y[0], region_z], 
                  [region_x[1], region_y[1], region_z+1]]
rectangle_05 = [[big_05_x[0], big_05_y[0], region_z], 
                  [big_05_x[1], big_05_y[1], region_z+1]]
rectangle_06 = [[big_06_x[0], big_06_y[0], region_z], 
                  [big_06_x[1], big_06_y[1], region_z+1]]
rectangle_08 = [[big_x[0], big_y[0], region_z], 
                  [big_x[1], big_y[1], region_z+1]]
rectangle_extend = [[extend_x[0], extend_y[0], region_z], 
                  [extend_x[1], extend_y[1], region_z+1]]
rectangle_extend2 = [[extend2_x[0], extend2_y[0], region_z], 
                  [extend2_x[1], extend2_y[1], region_z+1]]

# Make region images, needed for simulations only
region_data = np.zeros((600, 500, 50), dtype=np.int16)
region_data[region_x[0]: region_x[1], region_y[0]: region_y[1], region_z] = region_id
region = pyto.segmentation.Labels(data=region_data)

region_extend_data = np.zeros((1600, 200, 50), dtype=np.int16)
region_extend_data[extend_x[0]: extend_x[1], extend_y[0]: extend_y[1], region_z] = region_id
region_extend = pyto.segmentation.Labels(data=region_extend_data)

region_extend2_data = np.zeros((4000, 200, 50), dtype=np.int16)
region_extend2_data[extend2_x[0]: extend2_x[1], extend2_y[0]: extend2_y[1], region_z] = region_id
region_extend2 = pyto.segmentation.Labels(data=region_extend2_data)

# Make region coordinates
rectangle_full_coords = point_pattern.get_rectangle_coords(rectangle_full)
for size in ['50', '05', '06', '08']:
    exec(f"rectangle_{size}_coords = point_pattern.get_rectangle_coords(rectangle_{size})")

#### Initialize particle table or read an existing one

Important:  
* Do not execute if all required particle patterns have already been generated
* Otherwise, execute one or the other

In [5]:
# Initialize particle table

# Execute to generate particle patterns from scratch
# Particle patterns are generated in the next section

# set to True if region was changed
write_regions = True

# make flat regions
if write_regions:
     region.write(file=region_path)
     region_extend.write(file=region_extend_path)
     region_extend2.write(file=region_extend2_path)

# make tomos dataframe
mps = MultiParticleSets()
mps.tomos = pd.DataFrame(
    {mps.tomo_id_col: ['alpha', 'bravo', 'charlie'], 
     mps.region_col: [region_path, region_extend_path, region_extend2_path],
     mps.pixel_nm_col: pixel_size_nm})

## Make point patterns in rectangular regions

* Generates and saves patterns
* Execute this section only if new patterns need to be executed
* If patterns already exist, skip this section and read particles in the next section
* When executing this part, to be safe:
    * Restart the notebook
    * Run all cells before this section
    * Run all cells of this section in the order, once each cell (the first cell initializes new patterns object mps, the other cells update that object)
    * Check N particles to see whether some sets are double

#### Random patterns A and B (no interaction)

Important: Many of these are used for making interacting pattrens. Therefore, if these are modified or made again, the related interacting clusters have to be generated again

In [6]:
# Random and monovariate particle sets

# make point patterns
pat_A_random_full = point_pattern.random_rectangle(N=n_A, rectangle=rectangle_full, seed=seed)
pat_A_random_50 = point_pattern.random_rectangle(N=n_A, rectangle=rectangle_50, seed=seed+1)
pat_A_random_extend = point_pattern.random_rectangle(N=n_A, rectangle=rectangle_extend, seed=seed+6)
pat_A_random_extend2 = point_pattern.random_rectangle(N=n_A, rectangle=rectangle_extend2, seed=seed+6)
pat_A_random_excl_full = point_pattern.random_rectangle(
    N=n_A, rectangle=rectangle_full, exclusion=exclusion, seed=seed+2)
pat_B_random_full = point_pattern.random_rectangle(N=n_B, rectangle=rectangle_full, seed=seed+3)
pat_B_random_50 = point_pattern.random_rectangle(N=n_B, rectangle=rectangle_50, seed=seed+4)
pat_B_random_extend = point_pattern.random_rectangle(N=n_B, rectangle=rectangle_extend, seed=seed+7)
pat_B_random_extend2 = point_pattern.random_rectangle(N=n_B, rectangle=rectangle_extend2, seed=seed+7)
pat_B_random_excl_full = point_pattern.random_rectangle(
    N=n_B, rectangle=rectangle_full, exclusion=exclusion, seed=seed+5)
pat_A_random_08 = point_pattern.random_rectangle(N=n_A, rectangle=rectangle_08, seed=seed+40)
pat_B_random_08 = point_pattern.random_rectangle(N=n_B, rectangle=rectangle_08, seed=seed+41)

# assign particle set names to point patterns
patterns = {
    'ARandomFull': pat_A_random_full, 'ARandom50': pat_A_random_50,
    'ARandomExclFull': pat_A_random_excl_full,  'ARandom08': pat_A_random_08,
    'BRandomFull': pat_B_random_full, 'BRandom50': pat_B_random_50,
    'BRandomExclFull': pat_B_random_excl_full, 'BRandom08': pat_B_random_08}
patterns_extend = {
    'ARandomExtend': pat_A_random_extend, 'BRandomExtend': pat_B_random_extend}
patterns_extend2 = {
    'ARandomExtend2': pat_A_random_extend2, 'BRandomExtend2': pat_B_random_extend2}

# make particles dataframe
particles_local = mps.from_patterns(
    patterns=patterns, coord_cols=mps.coord_reg_frame_cols, tomo_id='alpha', 
    pixel_size_nm=pixel_size_nm, update=False)
mps.particles = particles_local
mps.from_patterns(
    patterns=patterns_extend, coord_cols=mps.coord_reg_frame_cols, tomo_id='bravo', 
    pixel_size_nm=pixel_size_nm, update=True)
mps.from_patterns(
    patterns=patterns_extend2, coord_cols=mps.coord_reg_frame_cols, tomo_id='charlie', 
    pixel_size_nm=pixel_size_nm, update=True)
particles_basic = mps.particles

In [7]:
# Translated clustered pattern

mps.translate_pattern(
    original='BRandom50', new='BRandom50tx25', distance=[25, 0, 0], 
    tomo_id='alpha', update=True)

#### A & B 05, 06, 08, full w/wo mode 3 interaction

Fraction of A and fraction of B interact  
Rest of A and B spread over their areas

In [8]:
# different size random hard clusters

# Overlap fraction:
#   - 06w - 06e: 0.8
#   - 06sw - 06ne: 0.7
#   - 05w - 05e: 0.56
#   - 05sw - 05ne: 0.49

# make centered
pat_A_random_06 = point_pattern.random_rectangle(N=n_A, rectangle=rectangle_06, seed=seed+42)
pat_B_random_06 = point_pattern.random_rectangle(N=n_B, rectangle=rectangle_06, seed=seed+43)
pat_A_random_05 = point_pattern.random_rectangle(N=n_A, rectangle=rectangle_05, seed=seed+44)
pat_B_random_05 = point_pattern.random_rectangle(N=n_B, rectangle=rectangle_05, seed=seed+45)
patterns = {
    'ARandom06': pat_A_random_06, 'ARandom05': pat_A_random_05,
    'BRandom06': pat_B_random_06, 'BRandom05': pat_B_random_05}
mps.from_patterns(
    patterns=patterns, coord_cols=mps.coord_reg_frame_cols, tomo_id='alpha', 
    pixel_size_nm=pixel_size_nm, update=True)

In [9]:
# translated patterns

# parameters
size_dist = [
    ('06', 'w', -30, 0), ('06', 'e', 30, 0), ('05', 'w', -55, 0), ('05', 'e', 55, 0), 
    ('06', 'sw', -30, -10), ('06', 'ne', 30, 10), ('05', 'sw', -55, -10), ('05', 'ne', 55, 10)]
update = True

# translate region coords: Defines variables with names like 'ARandom06w'  
print("Generated patterns:")
for pset in ['A', 'B']:
    for size, loc, dx, dy in size_dist:
        new = f'{pset}Random{size}{loc}'
        mps.translate_pattern(
            original=f'{pset}Random{size}', new=new, 
            distance=[dx, dy, 0], tomo_id='alpha', update=update)
        print(f'    {new}')

# translated rectangle coords: Defines variables with names like 'rectangle_06w_coords'  
print("Extracted coordinates:")
for size, loc, dx, dy in size_dist:
    coords_name = f"rectangle_{size}{loc}_coords "
    exec(
        coords_name
        + f" = rectangle_{size}_coords + np.array([{dx}, {dy}, 0]).reshape(1, -1)")
    print(f"    {coords_name}")

Generated patterns:
    ARandom06w
    ARandom06e
    ARandom05w
    ARandom05e
    ARandom06sw
    ARandom06ne
    ARandom05sw
    ARandom05ne
    BRandom06w
    BRandom06e
    BRandom05w
    BRandom05e
    BRandom06sw
    BRandom06ne
    BRandom05sw
    BRandom05ne
Extracted coordinates:
    rectangle_06w_coords 
    rectangle_06e_coords 
    rectangle_05w_coords 
    rectangle_05e_coords 
    rectangle_06sw_coords 
    rectangle_06ne_coords 
    rectangle_05sw_coords 
    rectangle_05ne_coords 


In [10]:
# generate partially overlaping colocalized patterns
cases = [
    ('06', 'w', '06', 'e', 1, 1, 10), ('06', 'w', '06', 'e', 0.7, 0.7, 10),
    ('06', 'w', '06', 'e', 0.3, 0.3, 10),
    ('06', 'sw', '06', 'ne', 1, 1, 10), ('06', 'sw', '06', 'ne', 0.7, 0.7, 10),
    ('06', 'sw', '06', 'ne', 0.3, 0.3, 10),
    ('05', 'w', '05', 'e', 1, 1, 10), ('05', 'w', '05', 'e', 0.7, 0.7, 10),
    ('05', 'w', '05', 'e', 0.3, 0.3, 10),
    ('05', 'sw', '05', 'ne', 1, 1, 10), ('05', 'sw', '05', 'ne', 0.7, 0.7, 10),
    ('05', 'sw', '05', 'ne', 0.3, 0.3, 10)]
fraction_format = {1: '1', 0.7: '07', 0.3: '03'}
print("Generated patterns:")
for ind, (size_fixed, loc_fixed, size_coloc, loc_coloc, p_fixed, p_coloc, dist) in enumerate(cases):
    exec(f"coloc_region_coords = rectangle_{size_coloc}{loc_coloc}_coords")
    col_name = (
            f"Br{size_coloc}{loc_coloc}CoclustAr{size_fixed}{loc_fixed}"
            + f"p{fraction_format[p_fixed]}-p{fraction_format[p_coloc]}-d{dist}")
    mps.colocalize_pattern(
        fixed_name=f'ARandom{size_fixed}{loc_fixed}', n_colocalize=n_B, 
        colocalize_name=col_name, region_coords=coloc_region_coords, fixed_fraction=p_fixed, 
        colocalize_fraction=p_coloc, max_dist=dist, update=True, seed=seed+300+2*ind)
    print(f"\t{col_name}")
    col_name = (
            f"Ar{size_coloc}{loc_coloc}CoclustBr{size_fixed}{loc_fixed}"
            + f"p{p_fixed}-p{p_coloc}-d{dist}")
    mps.colocalize_pattern(
        fixed_name=f'BRandom{size_fixed}{loc_fixed}', n_colocalize=n_A, 
        colocalize_name=col_name, region_coords=coloc_region_coords, fixed_fraction=p_fixed, 
        colocalize_fraction=p_coloc, max_dist=dist, update=True, seed=seed+300+(2*ind+1))
    print(f"\t{col_name}")
print(f"Seeds used: {seed + 300} - {seed+300+(2*ind+1)}")


Generated patterns:
	Br06eCoclustAr06wp1-p1-d10
	Ar06eCoclustBr06wp1-p1-d10
	Br06eCoclustAr06wp07-p07-d10
	Ar06eCoclustBr06wp0.7-p0.7-d10
	Br06eCoclustAr06wp03-p03-d10
	Ar06eCoclustBr06wp0.3-p0.3-d10
	Br06neCoclustAr06swp1-p1-d10
	Ar06neCoclustBr06swp1-p1-d10
	Br06neCoclustAr06swp07-p07-d10
	Ar06neCoclustBr06swp0.7-p0.7-d10
	Br06neCoclustAr06swp03-p03-d10
	Ar06neCoclustBr06swp0.3-p0.3-d10
	Br05eCoclustAr05wp1-p1-d10
	Ar05eCoclustBr05wp1-p1-d10
	Br05eCoclustAr05wp07-p07-d10
	Ar05eCoclustBr05wp0.7-p0.7-d10
	Br05eCoclustAr05wp03-p03-d10
	Ar05eCoclustBr05wp0.3-p0.3-d10
	Br05neCoclustAr05swp1-p1-d10
	Ar05neCoclustBr05swp1-p1-d10
	Br05neCoclustAr05swp07-p07-d10
	Ar05neCoclustBr05swp0.7-p0.7-d10
	Br05neCoclustAr05swp03-p03-d10
	Ar05neCoclustBr05swp0.3-p0.3-d10
Seeds used: 123756 - 123779


In [11]:
# generate full colocalized patterns
cases = [
    ('f', '', 'f', '', 1, 1, 10), ('f', '', 'f', '', 0.7, 0.7, 10),
    ('f', '', 'f', '', 0.3, 0.3, 10)
]
seed_start = seed + 340
fraction_format = {1: '1', 0.7: '07', 0.3: '03'}
print("Generated patterns:")
for ind, (size_fixed, loc_fixed, size_coloc, loc_coloc, p_fixed, p_coloc, dist) in enumerate(cases):
    exec(f"coloc_region_coords = rectangle_full{loc_coloc}_coords")
    col_name = (
            f"Br{size_coloc}{loc_coloc}CoclustAr{size_fixed}{loc_fixed}"
            + f"p{fraction_format[p_fixed]}-p{fraction_format[p_coloc]}-d{dist}")
    mps.colocalize_pattern(
        fixed_name=f'ARandomFull{loc_fixed}', n_colocalize=n_B, 
        colocalize_name=col_name, region_coords=coloc_region_coords, fixed_fraction=p_fixed, 
        colocalize_fraction=p_coloc, max_dist=dist, update=True, seed=seed_start+2*ind)
    print(f"\t{col_name}")
    col_name = (
            f"Ar{size_coloc}{loc_coloc}CoclustBr{size_fixed}{loc_fixed}"
            + f"p{p_fixed}-p{p_coloc}-d{dist}")
    mps.colocalize_pattern(
        fixed_name=f'BRandomFull{loc_fixed}', n_colocalize=n_A, 
        colocalize_name=col_name, region_coords=coloc_region_coords, fixed_fraction=p_fixed, 
        colocalize_fraction=p_coloc, max_dist=dist, update=True, seed=seed_start+(2*ind+1))
    print(f"\t{col_name}")
print(f"Seeds used: {seed_start} - {seed_start+(2*ind+1)}")


Generated patterns:
	BrfCoclustArfp1-p1-d10
	ArfCoclustBrfp1-p1-d10
	BrfCoclustArfp07-p07-d10
	ArfCoclustBrfp0.7-p0.7-d10
	BrfCoclustArfp03-p03-d10
	ArfCoclustBrfp0.3-p0.3-d10
Seeds used: 123796 - 123801


In [12]:
# generate full colocalized patterns
cases = [
    ('f', '', 'f', '', 0.6, 0.6, 10), ('f', '', 'f', '', 0.5, 0.5, 10),
    ('f', '', 'f', '', 0.4, 0.4, 10)
]
seed_start = seed + 350
fraction_format = {0.6: '06', 0.5: '05', 0.4: '04'}
print("Generated patterns:")
for ind, (size_fixed, loc_fixed, size_coloc, loc_coloc, p_fixed, p_coloc, dist) in enumerate(cases):
    exec(f"coloc_region_coords = rectangle_full{loc_coloc}_coords")
    col_name = (
            f"Br{size_coloc}{loc_coloc}CoclustAr{size_fixed}{loc_fixed}"
            + f"p{fraction_format[p_fixed]}-p{fraction_format[p_coloc]}-d{dist}")
    mps.colocalize_pattern(
        fixed_name=f'ARandomFull{loc_fixed}', n_colocalize=n_B, 
        colocalize_name=col_name, region_coords=coloc_region_coords, fixed_fraction=p_fixed, 
        colocalize_fraction=p_coloc, max_dist=dist, update=True, seed=seed_start+2*ind)
    print(f"\t{col_name}")
    col_name = (
            f"Ar{size_coloc}{loc_coloc}CoclustBr{size_fixed}{loc_fixed}"
            + f"p{p_fixed}-p{p_coloc}-d{dist}")
    mps.colocalize_pattern(
        fixed_name=f'BRandomFull{loc_fixed}', n_colocalize=n_A, 
        colocalize_name=col_name, region_coords=coloc_region_coords, fixed_fraction=p_fixed, 
        colocalize_fraction=p_coloc, max_dist=dist, update=True, seed=seed_start+(2*ind+1))
    print(f"\t{col_name}")
print(f"Seeds used: {seed_start} - {seed_start+(2*ind+1)}")


Generated patterns:
	BrfCoclustArfp06-p06-d10
	ArfCoclustBrfp0.6-p0.6-d10
	BrfCoclustArfp05-p05-d10
	ArfCoclustBrfp0.5-p0.5-d10
	BrfCoclustArfp04-p04-d10
	ArfCoclustBrfp0.4-p0.4-d10
Seeds used: 123806 - 123811


In [13]:
# generate overlapping colocalized patterns
cases = [
    ('08', '', '08', '', 1, 1, 10), ('08', '', '08', '', 0.7, 0.7, 10),
    ('08', '', '08', '', 0.3, 0.3, 10),
    ('06', '', '06', '', 1, 1, 10), ('06', '', '06', '', 0.7, 0.7, 10),
    ('06', '', '06', '', 0.3, 0.3, 10),
    ('05', '', '05', '', 1, 1, 10), ('05', '', '05', '', 0.7, 0.7, 10),
    ('05', '', '05', '', 0.3, 0.3, 10),
    ]
fraction_format = {1: '1', 0.7: '07', 0.3: '03'}
seed_start = seed + 360
print("Generated patterns:")
for ind, (size_fixed, loc_fixed, size_coloc, loc_coloc, p_fixed, p_coloc, dist) in enumerate(cases):
    exec(f"coloc_region_coords = rectangle_{size_coloc}{loc_coloc}_coords")
    col_name = (
            f"Br{size_coloc}{loc_coloc}CoclustAr{size_fixed}{loc_fixed}"
            + f"p{fraction_format[p_fixed]}-p{fraction_format[p_coloc]}-d{dist}")
    mps.colocalize_pattern(
        fixed_name=f'ARandom{size_fixed}{loc_fixed}', n_colocalize=n_B, 
        colocalize_name=col_name, region_coords=coloc_region_coords, fixed_fraction=p_fixed, 
        colocalize_fraction=p_coloc, max_dist=dist, update=True, seed=seed_start+2*ind)
    print(f"\t{col_name}")
    col_name = (
            f"Ar{size_coloc}{loc_coloc}CoclustBr{size_fixed}{loc_fixed}"
            + f"p{p_fixed}-p{p_coloc}-d{dist}")
    mps.colocalize_pattern(
        fixed_name=f'BRandom{size_fixed}{loc_fixed}', n_colocalize=n_A, 
        colocalize_name=col_name, region_coords=coloc_region_coords, fixed_fraction=p_fixed, 
        colocalize_fraction=p_coloc, max_dist=dist, update=True, seed=seed_start+(2*ind+1))
    print(f"\t{col_name}")
print(f"Seeds used: {seed_start} - {seed_start+(2*ind+1)}")


Generated patterns:
	Br08CoclustAr08p1-p1-d10
	Ar08CoclustBr08p1-p1-d10
	Br08CoclustAr08p07-p07-d10
	Ar08CoclustBr08p0.7-p0.7-d10
	Br08CoclustAr08p03-p03-d10
	Ar08CoclustBr08p0.3-p0.3-d10
	Br06CoclustAr06p1-p1-d10
	Ar06CoclustBr06p1-p1-d10
	Br06CoclustAr06p07-p07-d10
	Ar06CoclustBr06p0.7-p0.7-d10
	Br06CoclustAr06p03-p03-d10
	Ar06CoclustBr06p0.3-p0.3-d10
	Br05CoclustAr05p1-p1-d10
	Ar05CoclustBr05p1-p1-d10
	Br05CoclustAr05p07-p07-d10
	Ar05CoclustBr05p0.7-p0.7-d10
	Br05CoclustAr05p03-p03-d10
	Ar05CoclustBr05p0.3-p0.3-d10
Seeds used: 123816 - 123833


In [14]:
# generate overlapping colocalized patterns d=5
cases = [
    ('08', '', '08', '', 1, 1, 5), ('08', '', '08', '', 0.7, 0.7, 5),
    ('08', '', '08', '', 0.3, 0.3, 5),
    ('06', '', '06', '', 1, 1, 5), ('06', '', '06', '', 0.7, 0.7, 5),
    ('06', '', '06', '', 0.3, 0.3, 5),
    ('05', '', '05', '', 1, 1, 5), ('05', '', '05', '', 0.7, 0.7, 5),
    ('05', '', '05', '', 0.3, 0.3, 5),
    ]
fraction_format = {1: '1', 0.7: '07', 0.3: '03'}
seed_start = seed + 400
print("Generated patterns:")
for ind, (size_fixed, loc_fixed, size_coloc, loc_coloc, p_fixed, p_coloc, dist) in enumerate(cases):
    exec(f"coloc_region_coords = rectangle_{size_coloc}{loc_coloc}_coords")
    col_name = (
            f"Br{size_coloc}{loc_coloc}CoclustAr{size_fixed}{loc_fixed}"
            + f"p{fraction_format[p_fixed]}-p{fraction_format[p_coloc]}-d{dist}")
    mps.colocalize_pattern(
        fixed_name=f'ARandom{size_fixed}{loc_fixed}', n_colocalize=n_B, 
        colocalize_name=col_name, region_coords=coloc_region_coords, fixed_fraction=p_fixed, 
        colocalize_fraction=p_coloc, max_dist=dist, update=True, seed=seed_start+2*ind)
    print(f"\t{col_name}")
    col_name = (
            f"Ar{size_coloc}{loc_coloc}CoclustBr{size_fixed}{loc_fixed}"
            + f"p{p_fixed}-p{p_coloc}-d{dist}")
    mps.colocalize_pattern(
        fixed_name=f'BRandom{size_fixed}{loc_fixed}', n_colocalize=n_A, 
        colocalize_name=col_name, region_coords=coloc_region_coords, fixed_fraction=p_fixed, 
        colocalize_fraction=p_coloc, max_dist=dist, update=True, seed=seed_start+(2*ind+1))
    print(f"\t{col_name}")
print(f"Seeds used: {seed_start} - {seed_start+(2*ind+1)}")


Generated patterns:
	Br08CoclustAr08p1-p1-d5
	Ar08CoclustBr08p1-p1-d5
	Br08CoclustAr08p07-p07-d5
	Ar08CoclustBr08p0.7-p0.7-d5
	Br08CoclustAr08p03-p03-d5
	Ar08CoclustBr08p0.3-p0.3-d5
	Br06CoclustAr06p1-p1-d5
	Ar06CoclustBr06p1-p1-d5
	Br06CoclustAr06p07-p07-d5
	Ar06CoclustBr06p0.7-p0.7-d5
	Br06CoclustAr06p03-p03-d5
	Ar06CoclustBr06p0.3-p0.3-d5
	Br05CoclustAr05p1-p1-d5
	Ar05CoclustBr05p1-p1-d5
	Br05CoclustAr05p07-p07-d5
	Ar05CoclustBr05p0.7-p0.7-d5
	Br05CoclustAr05p03-p03-d5
	Ar05CoclustBr05p0.3-p0.3-d5
Seeds used: 123856 - 123873


In [15]:
# generate partially overlaping colocalized patterns d=5
cases = [
    ('06', 'w', '06', 'e', 1, 1, 5), ('06', 'w', '06', 'e', 0.7, 0.7, 5),
    ('06', 'w', '06', 'e', 0.3, 0.3, 5),
    ('06', 'sw', '06', 'ne', 1, 1, 5), ('06', 'sw', '06', 'ne', 0.7, 0.7, 5),
    ('06', 'sw', '06', 'ne', 0.3, 0.3, 5),
    ('05', 'w', '05', 'e', 1, 1, 5), ('05', 'w', '05', 'e', 0.7, 0.7, 5),
    ('05', 'w', '05', 'e', 0.3, 0.3, 5),
    ('05', 'sw', '05', 'ne', 1, 1, 5), ('05', 'sw', '05', 'ne', 0.7, 0.7, 5),
    ('05', 'sw', '05', 'ne', 0.3, 0.3, 5)]
seed_start = seed + 440
fraction_format = {1: '1', 0.7: '07', 0.3: '03'}
print("Generated patterns:")
for ind, (size_fixed, loc_fixed, size_coloc, loc_coloc, p_fixed, p_coloc, dist) in enumerate(cases):
    exec(f"coloc_region_coords = rectangle_{size_coloc}{loc_coloc}_coords")
    col_name = (
            f"Br{size_coloc}{loc_coloc}CoclustAr{size_fixed}{loc_fixed}"
            + f"p{fraction_format[p_fixed]}-p{fraction_format[p_coloc]}-d{dist}")
    mps.colocalize_pattern(
        fixed_name=f'ARandom{size_fixed}{loc_fixed}', n_colocalize=n_B, 
        colocalize_name=col_name, region_coords=coloc_region_coords, fixed_fraction=p_fixed, 
        colocalize_fraction=p_coloc, max_dist=dist, update=True, seed=seed_start+2*ind)
    print(f"\t{col_name}")
    col_name = (
            f"Ar{size_coloc}{loc_coloc}CoclustBr{size_fixed}{loc_fixed}"
            + f"p{p_fixed}-p{p_coloc}-d{dist}")
    mps.colocalize_pattern(
        fixed_name=f'BRandom{size_fixed}{loc_fixed}', n_colocalize=n_A, 
        colocalize_name=col_name, region_coords=coloc_region_coords, fixed_fraction=p_fixed, 
        colocalize_fraction=p_coloc, max_dist=dist, update=True, seed=seed_start+(2*ind+1))
    print(f"\t{col_name}")
print(f"Seeds used: {seed_start} - {seed_start+(2*ind+1)}")


Generated patterns:
	Br06eCoclustAr06wp1-p1-d5
	Ar06eCoclustBr06wp1-p1-d5
	Br06eCoclustAr06wp07-p07-d5
	Ar06eCoclustBr06wp0.7-p0.7-d5
	Br06eCoclustAr06wp03-p03-d5
	Ar06eCoclustBr06wp0.3-p0.3-d5
	Br06neCoclustAr06swp1-p1-d5
	Ar06neCoclustBr06swp1-p1-d5
	Br06neCoclustAr06swp07-p07-d5
	Ar06neCoclustBr06swp0.7-p0.7-d5
	Br06neCoclustAr06swp03-p03-d5
	Ar06neCoclustBr06swp0.3-p0.3-d5
	Br05eCoclustAr05wp1-p1-d5
	Ar05eCoclustBr05wp1-p1-d5
	Br05eCoclustAr05wp07-p07-d5
	Ar05eCoclustBr05wp0.7-p0.7-d5
	Br05eCoclustAr05wp03-p03-d5
	Ar05eCoclustBr05wp0.3-p0.3-d5
	Br05neCoclustAr05swp1-p1-d5
	Ar05neCoclustBr05swp1-p1-d5
	Br05neCoclustAr05swp07-p07-d5
	Ar05neCoclustBr05swp0.7-p0.7-d5
	Br05neCoclustAr05swp03-p03-d5
	Ar05neCoclustBr05swp0.3-p0.3-d5
Seeds used: 123896 - 123919


In [16]:
# B colocalized with the A mode 1-to-1, d=15

# generate B interacting with A patterns
cases = [
    ('f', '', 'f', '', 0.7, 'kd', 15),
    ('f', '', 'f', '', 0.3, 'kd', 15),
    ('f', '', 'f', '', 0.2, 'kd', 15),
    ('f', '', 'f', '', 0.1, 'kd', 15),
    ('08', '', '08', '', 0.7, 'kd', 15),
    ('08', '', '08', '', 0.3, 'kd', 15),
    ('08', '', '08', '', 0.2, 'kd', 15),
    ('08', '', '08', '', 0.1, 'kd', 15),
    ('06', '', '06', '', 0.7, 'kd', 15),
    ('06', '', '06', '', 0.3, 'kd', 15),
    ('06', '', '06', '', 0.2, 'kd', 15),
    ('06', '', '06', '', 0.1, 'kd', 15),
    ('05', '', '05', '', 0.7, 'kd', 15),
    ('05', '', '05', '', 0.3, 'kd', 15),
    ('05', '', '05', '', 0.2, 'kd', 15),
    ('05', '', '05', '', 0.1, 'kd', 15)
]
seed_start = seed + 800
size_format = {'f': 'Full'}
size_format_region = {'f': 'full'}
fraction_format = {1: '1', 0.7: '07', 0.3: '03', 0.2: '02', 0.1: '01', 'kd': 'kd'}
print("Generated patterns:")
for ind, (size_fixed, loc_fixed, size_coloc, loc_coloc, p_fixed, p_coloc, dist) in enumerate(cases):
    size_coloc_region_formatted = size_format_region.get(size_coloc)
    if size_coloc_region_formatted is None:
        size_coloc_region_formatted = size_coloc
    exec(f"coloc_region_coords = rectangle_{size_coloc_region_formatted}{loc_coloc}_coords")
    col_name = (
            f"Br{size_coloc}{loc_coloc}CoclustAr{size_fixed}{loc_fixed}"
            + f"p{fraction_format[p_fixed]}-p{fraction_format[p_coloc]}-d{dist}")
    size_coloc_formatted = size_format.get(size_coloc)
    if size_coloc_formatted is None:
        size_coloc_formatted = size_coloc
    fixed_name = f'ARandom{size_coloc_formatted}{loc_fixed}'
    mps.colocalize_pattern(
        fixed_name=fixed_name, n_colocalize=n_B, mode='kd', keep_fixed=False,
        colocalize_name=col_name, region_coords=coloc_region_coords, fixed_fraction=p_fixed, 
        colocalize_fraction=p_coloc, max_dist=dist, update=True, seed=seed_start+2*ind)
    print(f"\t{col_name}  (coloc with {fixed_name})")
print(f"Seeds used: {seed_start} - {seed_start+(2*ind+1)}")

Generated patterns:
	BrfCoclustArfp07-pkd-d15  (coloc with ARandomFull)
	BrfCoclustArfp03-pkd-d15  (coloc with ARandomFull)
	BrfCoclustArfp02-pkd-d15  (coloc with ARandomFull)
	BrfCoclustArfp01-pkd-d15  (coloc with ARandomFull)
	Br08CoclustAr08p07-pkd-d15  (coloc with ARandom08)
	Br08CoclustAr08p03-pkd-d15  (coloc with ARandom08)
	Br08CoclustAr08p02-pkd-d15  (coloc with ARandom08)
	Br08CoclustAr08p01-pkd-d15  (coloc with ARandom08)
	Br06CoclustAr06p07-pkd-d15  (coloc with ARandom06)
	Br06CoclustAr06p03-pkd-d15  (coloc with ARandom06)
	Br06CoclustAr06p02-pkd-d15  (coloc with ARandom06)
	Br06CoclustAr06p01-pkd-d15  (coloc with ARandom06)
	Br05CoclustAr05p07-pkd-d15  (coloc with ARandom05)
	Br05CoclustAr05p03-pkd-d15  (coloc with ARandom05)
	Br05CoclustAr05p02-pkd-d15  (coloc with ARandom05)
	Br05CoclustAr05p01-pkd-d15  (coloc with ARandom05)
Seeds used: 124256 - 124287


#### Pattern C

In [17]:
# C random particle sets

# random
pat_C_random_full = point_pattern.random_rectangle(N=n_C, rectangle=rectangle_full, seed=seed+50)
patterns_c = {'CRandomFull': pat_C_random_full}

mps.from_patterns(
    patterns=patterns_c, coord_cols=mps.coord_reg_frame_cols, tomo_id='alpha', 
    pixel_size_nm=pixel_size_nm, update=True)

In [18]:
# make non-interacting (random) C patterns

patterns = {
    'CRandom08': point_pattern.random_rectangle(N=n_C, rectangle=rectangle_08, seed=seed+51),
    'CRandom06': point_pattern.random_rectangle(N=n_C, rectangle=rectangle_06, seed=seed+52),
    'CRandom05': point_pattern.random_rectangle(N=n_C, rectangle=rectangle_05, seed=seed+53)
    }
mps.from_patterns(
    patterns=patterns, coord_cols=mps.coord_reg_frame_cols, tomo_id='alpha', 
    pixel_size_nm=pixel_size_nm, update=True)

In [19]:
# generate C interacting with A patterns
cases = [
    ('f', '', 'f', '', 1, 1, 10), ('f', '', 'f', '', 0.7, 0.7, 10),
    ('f', '', 'f', '', 0.3, 0.3, 10),
    ('08', '', '08', '', 1, 1, 10), ('08', '', '08', '', 0.7, 0.7, 10),
    ('08', '', '08', '', 0.3, 0.3, 10),
    ('06', '', '06', '', 1, 1, 10), ('06', '', '06', '', 0.7, 0.7, 10),
    ('06', '', '06', '', 0.3, 0.3, 10),
    ('05', '', '05', '', 1, 1, 10), ('05', '', '05', '', 0.7, 0.7, 10),
    ('05', '', '05', '', 0.3, 0.3, 10)
]
seed_start = seed + 500
size_format = {'f': 'full'}
fraction_format = {1: '1', 0.7: '07', 0.3: '03'}
print("Generated patterns:")
for ind, (size_fixed, loc_fixed, size_coloc, loc_coloc, p_fixed, p_coloc, dist) in enumerate(cases):
    size_coloc_formatted = size_format.get(size_coloc)
    if size_coloc_formatted is None:
        size_coloc_formatted = size_coloc
    exec(f"coloc_region_coords = rectangle_{size_coloc_formatted}{loc_coloc}_coords")
    col_name = (
            f"Cr{size_coloc}{loc_coloc}CoclustAr{size_fixed}{loc_fixed}"
            + f"p{fraction_format[p_fixed]}-p{fraction_format[p_coloc]}-d{dist}")
    mps.colocalize_pattern(
        fixed_name=f'ARandomFull{loc_fixed}', n_colocalize=n_C, 
        colocalize_name=col_name, region_coords=coloc_region_coords, fixed_fraction=p_fixed, 
        colocalize_fraction=p_coloc, max_dist=dist, update=True, seed=seed_start+2*ind)
    print(f"\t{col_name}")
    col_name = (
            f"Ar{size_coloc}{loc_coloc}CoclustCr{size_fixed}{loc_fixed}"
            + f"p{fraction_format[p_fixed]}-p{fraction_format[p_coloc]}-d{dist}")
    mps.colocalize_pattern(
        fixed_name=f'CRandomFull{loc_fixed}', n_colocalize=n_A, 
        colocalize_name=col_name, region_coords=coloc_region_coords, fixed_fraction=p_fixed, 
        colocalize_fraction=p_coloc, max_dist=dist, update=True, seed=seed_start+(2*ind+1))
    print(f"\t{col_name}")
print(f"Seeds used: {seed_start} - {seed_start+(2*ind+1)}")


Generated patterns:
	CrfCoclustArfp1-p1-d10
	ArfCoclustCrfp1-p1-d10
	CrfCoclustArfp07-p07-d10
	ArfCoclustCrfp07-p07-d10
	CrfCoclustArfp03-p03-d10
	ArfCoclustCrfp03-p03-d10
	Cr08CoclustAr08p1-p1-d10
	Ar08CoclustCr08p1-p1-d10
	Cr08CoclustAr08p07-p07-d10
	Ar08CoclustCr08p07-p07-d10
	Cr08CoclustAr08p03-p03-d10
	Ar08CoclustCr08p03-p03-d10
	Cr06CoclustAr06p1-p1-d10
	Ar06CoclustCr06p1-p1-d10
	Cr06CoclustAr06p07-p07-d10
	Ar06CoclustCr06p07-p07-d10
	Cr06CoclustAr06p03-p03-d10
	Ar06CoclustCr06p03-p03-d10
	Cr05CoclustAr05p1-p1-d10
	Ar05CoclustCr05p1-p1-d10
	Cr05CoclustAr05p07-p07-d10
	Ar05CoclustCr05p07-p07-d10
	Cr05CoclustAr05p03-p03-d10
	Ar05CoclustCr05p03-p03-d10
Seeds used: 123956 - 123979


#### Patterns B and C coloc with A

In [20]:
# B and C colocalized with the same subset of A

# generate B and C interacting with A patterns
cases = [
    ('f', '', 'f', '', 1, 1, 10), 
    ('f', '', 'f', '', 0.7, 0.7, 10),
    ('f', '', 'f', '', 0.3, 0.3, 10),
    ('08', '', '08', '', 1, 1, 10), ('08', '', '08', '', 0.7, 0.7, 10),
    ('08', '', '08', '', 0.3, 0.3, 10),
    ('06', '', '06', '', 1, 1, 10), ('06', '', '06', '', 0.7, 0.7, 10),
    ('06', '', '06', '', 0.3, 0.3, 10),
    ('05', '', '05', '', 1, 1, 10), ('05', '', '05', '', 0.7, 0.7, 10),
    ('05', '', '05', '', 0.3, 0.3, 10)
]
keep_fixed = True
keep_fixed_label = 'k' if keep_fixed else ''
seed_start = seed + 550
size_format = {'f': 'Full'}
size_format_region = {'f': 'full'}
fraction_format = {1: '1', 0.7: '07', 0.3: '03', 0.2: '02', 0.1: '01'}
print("Generated patterns:")
for ind, (size_fixed, loc_fixed, size_coloc, loc_coloc, p_fixed, p_coloc, dist) in enumerate(cases):
    size_coloc_region_formatted = size_format_region.get(size_coloc)
    if size_coloc_region_formatted is None:
        size_coloc_region_formatted = size_coloc
    exec(f"coloc_region_coords = rectangle_{size_coloc_region_formatted}{loc_coloc}_coords")
    col_name = (
            f"Br{size_coloc}{loc_coloc}CoclustAr{size_fixed}{loc_fixed}"
            + f"p{fraction_format[p_fixed]}{keep_fixed_label}-p{fraction_format[p_coloc]}-d{dist}")
    size_coloc_formatted = size_format.get(size_coloc)
    if size_coloc_formatted is None:
        size_coloc_formatted = size_coloc
    fixed_name = f'ARandom{size_coloc_formatted}{loc_fixed}'    
    mps.colocalize_pattern(
        fixed_name=fixed_name, n_colocalize=n_B, keep_fixed=True,
        colocalize_name=col_name, region_coords=coloc_region_coords, fixed_fraction=p_fixed, 
        colocalize_fraction=p_coloc, max_dist=dist, update=True, seed=seed_start+2*ind)
    print(f"\t{col_name}  (coloc with {fixed_name})")
    col_name = (
            f"Cr{size_coloc}{loc_coloc}CoclustAr{size_fixed}{loc_fixed}"
            + f"p{fraction_format[p_fixed]}{keep_fixed_label}-p{fraction_format[p_coloc]}-d{dist}")
    fixed_name = f'ARandom{size_coloc_formatted}{loc_fixed}'    
    mps.colocalize_pattern(
        fixed_name=fixed_name, n_colocalize=n_C, keep_fixed=True,
        colocalize_name=col_name, region_coords=coloc_region_coords, fixed_fraction=p_fixed, 
        colocalize_fraction=p_coloc, max_dist=dist, update=True, seed=seed_start+2*ind)
    print(f"\t{col_name}  (coloc with {fixed_name})")
print(f"Seeds used: {seed_start} - {seed_start+(2*ind+1)}")


Generated patterns:
	BrfCoclustArfp1k-p1-d10  (coloc with ARandomFull)
	CrfCoclustArfp1k-p1-d10  (coloc with ARandomFull)
	BrfCoclustArfp07k-p07-d10  (coloc with ARandomFull)
	CrfCoclustArfp07k-p07-d10  (coloc with ARandomFull)
	BrfCoclustArfp03k-p03-d10  (coloc with ARandomFull)
	CrfCoclustArfp03k-p03-d10  (coloc with ARandomFull)
	Br08CoclustAr08p1k-p1-d10  (coloc with ARandom08)
	Cr08CoclustAr08p1k-p1-d10  (coloc with ARandom08)
	Br08CoclustAr08p07k-p07-d10  (coloc with ARandom08)
	Cr08CoclustAr08p07k-p07-d10  (coloc with ARandom08)
	Br08CoclustAr08p03k-p03-d10  (coloc with ARandom08)
	Cr08CoclustAr08p03k-p03-d10  (coloc with ARandom08)
	Br06CoclustAr06p1k-p1-d10  (coloc with ARandom06)
	Cr06CoclustAr06p1k-p1-d10  (coloc with ARandom06)
	Br06CoclustAr06p07k-p07-d10  (coloc with ARandom06)
	Cr06CoclustAr06p07k-p07-d10  (coloc with ARandom06)
	Br06CoclustAr06p03k-p03-d10  (coloc with ARandom06)
	Cr06CoclustAr06p03k-p03-d10  (coloc with ARandom06)
	Br05CoclustAr05p1k-p1-d10  (coloc wit

In [21]:
# generate full colocalized patterns
cases = [
    ('f', '', 'f', '', 0.6, 0.6, 10), ('f', '', 'f', '', 0.5, 0.5, 10),
    ('f', '', 'f', '', 0.4, 0.4, 10)
]
seed_start = seed + 570
fraction_format = {0.6: '06', 0.5: '05', 0.4: '04'}
print("Generated patterns:")
for ind, (size_fixed, loc_fixed, size_coloc, loc_coloc, p_fixed, p_coloc, dist) in enumerate(cases):
    exec(f"coloc_region_coords = rectangle_full{loc_coloc}_coords")
    col_name = (
            f"Cr{size_coloc}{loc_coloc}CoclustAr{size_fixed}{loc_fixed}"
            + f"p{fraction_format[p_fixed]}-p{fraction_format[p_coloc]}-d{dist}")
    mps.colocalize_pattern(
        fixed_name=f'ARandomFull{loc_fixed}', n_colocalize=n_C, 
        colocalize_name=col_name, region_coords=coloc_region_coords, fixed_fraction=p_fixed, 
        colocalize_fraction=p_coloc, max_dist=dist, update=True, seed=seed_start+2*ind)
    print(f"\t{col_name}")
    col_name = (
            f"Ar{size_coloc}{loc_coloc}CoclustCr{size_fixed}{loc_fixed}"
            + f"p{p_fixed}-p{p_coloc}-d{dist}")
    mps.colocalize_pattern(
        fixed_name=f'CRandomFull{loc_fixed}', n_colocalize=n_A, 
        colocalize_name=col_name, region_coords=coloc_region_coords, fixed_fraction=p_fixed, 
        colocalize_fraction=p_coloc, max_dist=dist, update=True, seed=seed_start+(2*ind+1))
    print(f"\t{col_name}")
print(f"Seeds used: {seed_start} - {seed_start+(2*ind+1)}")


Generated patterns:
	CrfCoclustArfp06-p06-d10
	ArfCoclustCrfp0.6-p0.6-d10
	CrfCoclustArfp05-p05-d10
	ArfCoclustCrfp0.5-p0.5-d10
	CrfCoclustArfp04-p04-d10
	ArfCoclustCrfp0.4-p0.4-d10
Seeds used: 124026 - 124031


In [22]:
# B colocalized with the A mode 1-to-1, d=10

# generate B interacting with A patterns
cases = [
    ('f', '', 'f', '', 0.7, 'kd', 10),
    ('f', '', 'f', '', 0.3, 'kd', 10),
    ('f', '', 'f', '', 0.2, 'kd', 10),
    ('f', '', 'f', '', 0.15, 'kd', 10),
    ('f', '', 'f', '', 0.1, 'kd', 10),
    ('f', '', 'f', '', 0.05, 'kd', 10),
    ('08', '', '08', '', 0.7, 'kd', 10),
    ('08', '', '08', '', 0.3, 'kd', 10),
    ('08', '', '08', '', 0.2, 'kd', 10),
    ('08', '', '08', '', 0.15, 'kd', 10),
    ('08', '', '08', '', 0.1, 'kd', 10),
    ('08', '', '08', '', 0.05, 'kd', 10),
    ('06', '', '06', '', 0.7, 'kd', 10),
    ('06', '', '06', '', 0.3, 'kd', 10),
    ('06', '', '06', '', 0.2, 'kd', 10),
    ('06', '', '06', '', 0.15, 'kd', 10),
    ('06', '', '06', '', 0.1, 'kd', 10),
    ('06', '', '06', '', 0.05, 'kd', 10),
    ('05', '', '05', '', 0.7, 'kd', 10),
    ('05', '', '05', '', 0.3, 'kd', 10),
    ('05', '', '05', '', 0.2, 'kd', 10),
    ('05', '', '05', '', 0.15, 'kd', 10),
    ('05', '', '05', '', 0.1, 'kd', 10),
    ('05', '', '05', '', 0.05, 'kd', 10)
]
seed_start = seed + 600
size_format = {'f': 'Full'}
size_format_region = {'f': 'full'}
fraction_format = {
    1: '1', 0.7: '07', 0.3: '03', 0.2: '02', 0.15: '015', 0.1: '01', 0.05: '005', 'kd': 'kd'}
print("Generated patterns:")
for ind, (size_fixed, loc_fixed, size_coloc, loc_coloc, p_fixed, p_coloc, dist) in enumerate(cases):
    size_coloc_region_formatted = size_format_region.get(size_coloc)
    if size_coloc_region_formatted is None:
        size_coloc_region_formatted = size_coloc
    exec(f"coloc_region_coords = rectangle_{size_coloc_region_formatted}{loc_coloc}_coords")
    col_name = (
            f"Br{size_coloc}{loc_coloc}CoclustAr{size_fixed}{loc_fixed}"
            + f"p{fraction_format[p_fixed]}-p{fraction_format[p_coloc]}-d{dist}")
    size_coloc_formatted = size_format.get(size_coloc)
    if size_coloc_formatted is None:
        size_coloc_formatted = size_coloc
    fixed_name = f'ARandom{size_coloc_formatted}{loc_fixed}'
    mps.colocalize_pattern(
        fixed_name=fixed_name, n_colocalize=n_B, mode='kd', keep_fixed=False,
        colocalize_name=col_name, region_coords=coloc_region_coords, fixed_fraction=p_fixed, 
        colocalize_fraction=p_coloc, max_dist=dist, update=True, seed=seed_start+2*ind)
    print(f"\t{col_name}  (coloc with {fixed_name})")
print(f"Seeds used: {seed_start} - {seed_start+(2*ind+1)}")

Generated patterns:
	BrfCoclustArfp07-pkd-d10  (coloc with ARandomFull)
	BrfCoclustArfp03-pkd-d10  (coloc with ARandomFull)
	BrfCoclustArfp02-pkd-d10  (coloc with ARandomFull)
	BrfCoclustArfp015-pkd-d10  (coloc with ARandomFull)
	BrfCoclustArfp01-pkd-d10  (coloc with ARandomFull)
	BrfCoclustArfp005-pkd-d10  (coloc with ARandomFull)
	Br08CoclustAr08p07-pkd-d10  (coloc with ARandom08)
	Br08CoclustAr08p03-pkd-d10  (coloc with ARandom08)
	Br08CoclustAr08p02-pkd-d10  (coloc with ARandom08)
	Br08CoclustAr08p015-pkd-d10  (coloc with ARandom08)
	Br08CoclustAr08p01-pkd-d10  (coloc with ARandom08)
	Br08CoclustAr08p005-pkd-d10  (coloc with ARandom08)
	Br06CoclustAr06p07-pkd-d10  (coloc with ARandom06)
	Br06CoclustAr06p03-pkd-d10  (coloc with ARandom06)
	Br06CoclustAr06p02-pkd-d10  (coloc with ARandom06)
	Br06CoclustAr06p015-pkd-d10  (coloc with ARandom06)
	Br06CoclustAr06p01-pkd-d10  (coloc with ARandom06)
	Br06CoclustAr06p005-pkd-d10  (coloc with ARandom06)
	Br05CoclustAr05p07-pkd-d10  (coloc wi

In [23]:
# B colocalized with the A mode 1-to-1, d=5

# generate B interacting with A patterns
cases = [
    ('f', '', 'f', '', 0.7, 'kd', 5),
    ('f', '', 'f', '', 0.3, 'kd', 5),
    ('f', '', 'f', '', 0.2, 'kd', 5),
    ('f', '', 'f', '', 0.1, 'kd', 5),
    ('08', '', '08', '', 0.7, 'kd', 5),
    ('08', '', '08', '', 0.3, 'kd', 5),
    ('08', '', '08', '', 0.2, 'kd', 5),
    ('08', '', '08', '', 0.1, 'kd', 5),
    ('06', '', '06', '', 0.7, 'kd', 5),
    ('06', '', '06', '', 0.3, 'kd', 5),
    ('06', '', '06', '', 0.2, 'kd', 5),
    ('06', '', '06', '', 0.1, 'kd', 5),
    ('05', '', '05', '', 0.7, 'kd', 5),
    ('05', '', '05', '', 0.3, 'kd', 5),
    ('05', '', '05', '', 0.2, 'kd', 5),
    ('05', '', '05', '', 0.1, 'kd', 5),
    ('f', '', 'f', '', 0.15, 'kd', 5)  # extra, Seeds used: 124104 - 124105
]
seed_start = seed + 600
size_format = {'f': 'Full'}
size_format_region = {'f': 'full'}
fraction_format = {
    1: '1', 0.7: '07', 0.3: '03', 0.2: '02', 0.15: '015', 0.1: '01', 'kd': 'kd'}
print("Generated patterns:")
for ind, (size_fixed, loc_fixed, size_coloc, loc_coloc, p_fixed, p_coloc, dist) in enumerate(cases):
    size_coloc_region_formatted = size_format_region.get(size_coloc)
    if size_coloc_region_formatted is None:
        size_coloc_region_formatted = size_coloc
    exec(f"coloc_region_coords = rectangle_{size_coloc_region_formatted}{loc_coloc}_coords")
    col_name = (
            f"Br{size_coloc}{loc_coloc}CoclustAr{size_fixed}{loc_fixed}"
            + f"p{fraction_format[p_fixed]}-p{fraction_format[p_coloc]}-d{dist}")
    size_coloc_formatted = size_format.get(size_coloc)
    if size_coloc_formatted is None:
        size_coloc_formatted = size_coloc
    fixed_name = f'ARandom{size_coloc_formatted}{loc_fixed}'
    mps.colocalize_pattern(
        fixed_name=fixed_name, n_colocalize=n_B, mode='kd', keep_fixed=False,
        colocalize_name=col_name, region_coords=coloc_region_coords, fixed_fraction=p_fixed, 
        colocalize_fraction=p_coloc, max_dist=dist, update=True, seed=seed_start+2*ind)
    print(f"\t{col_name}  (coloc with {fixed_name})")
print(f"Seeds used: {seed_start} - {seed_start+(2*ind+1)}")

Generated patterns:
	BrfCoclustArfp07-pkd-d5  (coloc with ARandomFull)
	BrfCoclustArfp03-pkd-d5  (coloc with ARandomFull)
	BrfCoclustArfp02-pkd-d5  (coloc with ARandomFull)
	BrfCoclustArfp01-pkd-d5  (coloc with ARandomFull)
	Br08CoclustAr08p07-pkd-d5  (coloc with ARandom08)
	Br08CoclustAr08p03-pkd-d5  (coloc with ARandom08)
	Br08CoclustAr08p02-pkd-d5  (coloc with ARandom08)
	Br08CoclustAr08p01-pkd-d5  (coloc with ARandom08)
	Br06CoclustAr06p07-pkd-d5  (coloc with ARandom06)
	Br06CoclustAr06p03-pkd-d5  (coloc with ARandom06)
	Br06CoclustAr06p02-pkd-d5  (coloc with ARandom06)
	Br06CoclustAr06p01-pkd-d5  (coloc with ARandom06)
	Br05CoclustAr05p07-pkd-d5  (coloc with ARandom05)
	Br05CoclustAr05p03-pkd-d5  (coloc with ARandom05)
	Br05CoclustAr05p02-pkd-d5  (coloc with ARandom05)
	Br05CoclustAr05p01-pkd-d5  (coloc with ARandom05)
	BrfCoclustArfp015-pkd-d5  (coloc with ARandomFull)
Seeds used: 124056 - 124089


In [24]:
# C colocalized with the A, mode 1-to-1, d=10

# generate B interacting with A patterns
cases = [
    ('f', '', 'f', '', 0.15, 'kd', 10),
    ('f', '', 'f', '', 0.1, 'kd', 10),
    ('f', '', 'f', '', 0.05, 'kd', 10),
    ('08', '', '08', '', 0.15, 'kd', 10),
    ('08', '', '08', '', 0.1, 'kd', 10),
    ('08', '', '08', '', 0.05, 'kd', 10),
    ('06', '', '06', '', 0.15, 'kd', 10),
    ('06', '', '06', '', 0.1, 'kd', 10),
    ('06', '', '06', '', 0.05, 'kd', 10),
    ('05', '', '05', '', 0.15, 'kd', 10),
    ('05', '', '05', '', 0.1, 'kd', 10),
    ('05', '', '05', '', 0.05, 'kd', 10)
]
seed_start = seed + 700
size_format = {'f': 'Full'}
size_format_region = {'f': 'full'}
fraction_format = {
    1: '1', 0.7: '07', 0.3: '03', 0.2: '02', 0.15: '015', 0.1: '01', 0.05: '005', 'kd': 'kd'}
print("Generated patterns:")
for ind, (size_fixed, loc_fixed, size_coloc, loc_coloc, p_fixed, p_coloc, dist) in enumerate(cases):
    size_coloc_region_formatted = size_format_region.get(size_coloc)
    if size_coloc_region_formatted is None:
        size_coloc_region_formatted = size_coloc
    exec(f"coloc_region_coords = rectangle_{size_coloc_region_formatted}{loc_coloc}_coords")
    col_name = (
            f"Cr{size_coloc}{loc_coloc}CoclustAr{size_fixed}{loc_fixed}"
            + f"p{fraction_format[p_fixed]}-p{fraction_format[p_coloc]}-d{dist}")
    size_coloc_formatted = size_format.get(size_coloc)
    if size_coloc_formatted is None:
        size_coloc_formatted = size_coloc
    fixed_name = f'ARandom{size_coloc_formatted}{loc_fixed}'
    mps.colocalize_pattern(
        fixed_name=fixed_name, n_colocalize=n_C, mode='kd', keep_fixed=False,
        colocalize_name=col_name, region_coords=coloc_region_coords, fixed_fraction=p_fixed, 
        colocalize_fraction=p_coloc, max_dist=dist, update=True, seed=seed_start+2*ind)
    print(f"\t{col_name}  (coloc with {fixed_name})")
print(f"Seeds used: {seed_start} - {seed_start+(2*ind+1)}")

Generated patterns:
	CrfCoclustArfp015-pkd-d10  (coloc with ARandomFull)
	CrfCoclustArfp01-pkd-d10  (coloc with ARandomFull)
	CrfCoclustArfp005-pkd-d10  (coloc with ARandomFull)
	Cr08CoclustAr08p015-pkd-d10  (coloc with ARandom08)
	Cr08CoclustAr08p01-pkd-d10  (coloc with ARandom08)
	Cr08CoclustAr08p005-pkd-d10  (coloc with ARandom08)
	Cr06CoclustAr06p015-pkd-d10  (coloc with ARandom06)
	Cr06CoclustAr06p01-pkd-d10  (coloc with ARandom06)
	Cr06CoclustAr06p005-pkd-d10  (coloc with ARandom06)
	Cr05CoclustAr05p015-pkd-d10  (coloc with ARandom05)
	Cr05CoclustAr05p01-pkd-d10  (coloc with ARandom05)
	Cr05CoclustAr05p005-pkd-d10  (coloc with ARandom05)
Seeds used: 124156 - 124179


In [25]:
# B and C colocalized with the same subset of A, mode 1-to-1

# generate B and C interacting with A patterns
cases = [
    ('f', '', 'f', '', 0.15, 'kd', 10),
    ('f', '', 'f', '', 0.1, 'kd', 10),
    ('f', '', 'f', '', 0.05, 'kd', 10),
    ('08', '', '08', '', 0.15, 'kd', 10),
    ('08', '', '08', '', 0.1, 'kd', 10),
    ('08', '', '08', '', 0.05, 'kd', 10),
    ('06', '', '06', '', 0.15, 'kd', 10),
    ('06', '', '06', '', 0.1, 'kd', 10),
    ('06', '', '06', '', 0.05, 'kd', 10),
    ('05', '', '05', '', 0.15, 'kd', 10),
    ('05', '', '05', '', 0.1, 'kd', 10),
    ('05', '', '05', '', 0.05, 'kd', 10)
]
keep_fixed = True
keep_fixed_label = 'k' if keep_fixed else ''
seed_start = seed + 750
size_format = {'f': 'Full'}
size_format_region = {'f': 'full'}
fraction_format = {
    1: '1', 0.7: '07', 0.3: '03', 0.2: '02', 0.15: '015', 0.1: '01', 0.05: '005', 'kd': 'kd'}
print("Generated patterns:")
for ind, (size_fixed, loc_fixed, size_coloc, loc_coloc, p_fixed, p_coloc, dist) in enumerate(cases):
    size_coloc_region_formatted = size_format_region.get(size_coloc)
    if size_coloc_region_formatted is None:
        size_coloc_region_formatted = size_coloc
    exec(f"coloc_region_coords = rectangle_{size_coloc_region_formatted}{loc_coloc}_coords")
    col_name = (
            f"Br{size_coloc}{loc_coloc}CoclustAr{size_fixed}{loc_fixed}"
            + f"p{fraction_format[p_fixed]}{keep_fixed_label}-p{fraction_format[p_coloc]}-d{dist}")
    size_coloc_formatted = size_format.get(size_coloc)
    if size_coloc_formatted is None:
        size_coloc_formatted = size_coloc
    fixed_name = f'ARandom{size_coloc_formatted}{loc_fixed}'    
    mps.colocalize_pattern(
        fixed_name=fixed_name, n_colocalize=n_B, mode='kd', keep_fixed=True,
        colocalize_name=col_name, region_coords=coloc_region_coords, fixed_fraction=p_fixed, 
        colocalize_fraction=p_coloc, max_dist=dist, update=True, seed=seed_start+2*ind)
    print(f"\t{col_name}  (coloc with {fixed_name})")
    col_name = (
            f"Cr{size_coloc}{loc_coloc}CoclustAr{size_fixed}{loc_fixed}"
            + f"p{fraction_format[p_fixed]}{keep_fixed_label}-p{fraction_format[p_coloc]}-d{dist}")
    fixed_name = f'ARandom{size_coloc_formatted}{loc_fixed}'    
    mps.colocalize_pattern(
        fixed_name=fixed_name, n_colocalize=n_C, mode='kd', keep_fixed=True,
        colocalize_name=col_name, region_coords=coloc_region_coords, fixed_fraction=p_fixed, 
        colocalize_fraction=p_coloc, max_dist=dist, update=True, seed=seed_start+2*ind)
    print(f"\t{col_name}  (coloc with {fixed_name})")
print(f"Seeds used: {seed_start} - {seed_start+(2*ind+1)}")


Generated patterns:
	BrfCoclustArfp015k-pkd-d10  (coloc with ARandomFull)
	CrfCoclustArfp015k-pkd-d10  (coloc with ARandomFull)
	BrfCoclustArfp01k-pkd-d10  (coloc with ARandomFull)
	CrfCoclustArfp01k-pkd-d10  (coloc with ARandomFull)
	BrfCoclustArfp005k-pkd-d10  (coloc with ARandomFull)
	CrfCoclustArfp005k-pkd-d10  (coloc with ARandomFull)
	Br08CoclustAr08p015k-pkd-d10  (coloc with ARandom08)
	Cr08CoclustAr08p015k-pkd-d10  (coloc with ARandom08)
	Br08CoclustAr08p01k-pkd-d10  (coloc with ARandom08)
	Cr08CoclustAr08p01k-pkd-d10  (coloc with ARandom08)
	Br08CoclustAr08p005k-pkd-d10  (coloc with ARandom08)
	Cr08CoclustAr08p005k-pkd-d10  (coloc with ARandom08)
	Br06CoclustAr06p015k-pkd-d10  (coloc with ARandom06)
	Cr06CoclustAr06p015k-pkd-d10  (coloc with ARandom06)
	Br06CoclustAr06p01k-pkd-d10  (coloc with ARandom06)
	Cr06CoclustAr06p01k-pkd-d10  (coloc with ARandom06)
	Br06CoclustAr06p005k-pkd-d10  (coloc with ARandom06)
	Cr06CoclustAr06p005k-pkd-d10  (coloc with ARandom06)
	Br05CoclustAr

#### Write particles

In [26]:
# write particles
mps.write(mps_path)

Pickled  MPS object to particles/mps.pkl


## Additional point patterns

Execute this section if additional patterns are added

In [27]:
# Read an existing particle table

# Execute when additional particle patterns are added to the existing ones 

# Warning: Only new patterns should be generated in the next section. Generating a pattern 
# that already exist in the table will double the number of particles

# read existing
mps = MultiParticleSets.read(mps_path)

Read  MPS object particles/mps.pkl


In [28]:
# generate additional patterns here



In [29]:
# Show all patterns and number of particles 

n_part = mps.particles.groupby(['tomo_id', 'subclass_name'])['pixel_nm'].count()
n_part

tomo_id  subclass_name                   
alpha    ARandom05                           300
         ARandom05e                          300
         ARandom05ne                         300
         ARandom05sw                         300
         ARandom05w                          300
         ARandom06                           300
         ARandom06e                          300
         ARandom06ne                         300
         ARandom06sw                         300
         ARandom06w                          300
         ARandom08                           300
         ARandom50                           300
         ARandomExclFull                     300
         ARandomFull                         300
         Ar05CoclustBr05p0.3-p0.3-d10        300
         Ar05CoclustBr05p0.3-p0.3-d5         300
         Ar05CoclustBr05p0.7-p0.7-d10        300
         Ar05CoclustBr05p0.7-p0.7-d5         300
         Ar05CoclustBr05p1-p1-d10            300
         Ar05CoclustBr05p1-

In [30]:
# Check N particles in all patterns
if np.all([eval(f"n_{pattern[0]} == {value}") 
           for (tomo_id, pattern), value in n_part.items()]):
    print("Success: All patterns have correct number of particles")
else:
    print("ERROR: Some patterns have incorrect number of particles")

Success: All patterns have correct number of particles


In [31]:
# check if class and subclss columns have the same values
if (mps.particles[mps.class_name_col].to_numpy() 
    == mps.particles[mps.subclass_col].to_numpy()).all():
    print("Success: Class and subclss columns have the same values")
else:
    print("ERROR: Class and subclss columns do not have the same values")

Success: Class and subclss columns have the same values


In [32]:
# write particles
mps.write(mps_path)

Pickled  MPS object to particles/mps.pkl


## Read particles and glance at them

In [33]:
# read particles
# Has to be executed if the previous section was not executed

mps = MultiParticleSets.read(mps_path)

Read  MPS object particles/mps.pkl


In [34]:
# Show all patterns and number of particles 

n_part = mps.particles.groupby(['tomo_id', 'subclass_name'])['pixel_nm'].count()
n_part

tomo_id  subclass_name                   
alpha    ARandom05                           300
         ARandom05e                          300
         ARandom05ne                         300
         ARandom05sw                         300
         ARandom05w                          300
         ARandom06                           300
         ARandom06e                          300
         ARandom06ne                         300
         ARandom06sw                         300
         ARandom06w                          300
         ARandom08                           300
         ARandom50                           300
         ARandomExclFull                     300
         ARandomFull                         300
         Ar05CoclustBr05p0.3-p0.3-d10        300
         Ar05CoclustBr05p0.3-p0.3-d5         300
         Ar05CoclustBr05p0.7-p0.7-d10        300
         Ar05CoclustBr05p0.7-p0.7-d5         300
         Ar05CoclustBr05p1-p1-d10            300
         Ar05CoclustBr05p1-

In [35]:
# Check N particles in all patterns
if np.all([eval(f"n_{pattern[0]} == {value}") 
           for (tomo_id, pattern), value in n_part.items()]):
    print("Success: All patterns have correct number of particles")
else:
    print("ERROR: Some patterns have incorrect number of particles")

Success: All patterns have correct number of particles


In [36]:
mps.tomos

,tomo_id,region,pixel_nm
0,alpha,regions/flat_region.mrc,1
1,bravo,regions/flat_region_extend.mrc,1
2,charlie,regions/flat_region_extend2.mrc,1


In [37]:
mps.particles.head()

,tomo_id,pixel_nm,class_name,subclass_name,x_reg_frame,y_reg_frame,z_reg_frame,keep
0,alpha,1,ARandomFull,ARandomFull,139,327,20,True
1,alpha,1,ARandomFull,ARandomFull,426,276,20,True
2,alpha,1,ARandomFull,ARandomFull,249,209,20,True
3,alpha,1,ARandomFull,ARandomFull,125,391,20,True
4,alpha,1,ARandomFull,ARandomFull,188,381,20,True


In [38]:
mps.particles.tail()

,tomo_id,pixel_nm,class_name,subclass_name,x_reg_frame,y_reg_frame,z_reg_frame,keep
62945,alpha,1,Cr05CoclustAr05p005k-pkd-d10,Cr05CoclustAr05p005k-pkd-d10,415,370,20,True
62946,alpha,1,Cr05CoclustAr05p005k-pkd-d10,Cr05CoclustAr05p005k-pkd-d10,356,299,20,True
62947,alpha,1,Cr05CoclustAr05p005k-pkd-d10,Cr05CoclustAr05p005k-pkd-d10,259,330,20,True
62948,alpha,1,Cr05CoclustAr05p005k-pkd-d10,Cr05CoclustAr05p005k-pkd-d10,390,280,20,True
62949,alpha,1,Cr05CoclustAr05p005k-pkd-d10,Cr05CoclustAr05p005k-pkd-d10,287,345,20,True


## Calculate 2-colocalizations

### Hard cluster 50 and full

#### Standard

In [39]:
# Colocalization

# colocalization name
coloc_name = 'ARandomFull_BRandomFull'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: ARandomFull_BRandomFull
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/ARandomFull_BRandomFull/ARandomFull_BRandomFull.pkl
N particles initial (before exclusion):
    subclass_name  ARandomFull  BRandomFull
    tomo_id                                
    alpha                  300          250
    Total                  300          250
Pickled  MPS object to coloc/after_exclusion_particles/ARandomFull_BRandomFull/ARandomFull_BRandomFull.pkl
N particles after exclusion:
    subclass_name  ARandomFull  BRandomFull
    tomo_id                                
    alpha                  300          250
    Total         

In [40]:
# Colocalization

# colocalization name
coloc_name = 'BRandomFull_ARandomFull'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: BRandomFull_ARandomFull
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/BRandomFull_ARandomFull/BRandomFull_ARandomFull.pkl
N particles initial (before exclusion):
    subclass_name  ARandomFull  BRandomFull
    tomo_id                                
    alpha                  300          250
    Total                  300          250
Pickled  MPS object to coloc/after_exclusion_particles/BRandomFull_ARandomFull/BRandomFull_ARandomFull.pkl
N particles after exclusion:
    subclass_name  ARandomFull  BRandomFull
    tomo_id                                
    alpha                  300          250
    Total         

#### Extended

In [41]:
# Colocalization

# colocalization name
coloc_name = 'ARandomExtend2_BRandomExtend2'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: ARandomExtend2_BRandomExtend2
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/ARandomExtend2_BRandomExtend2/ARandomExtend2_BRandomExtend2.pkl
N particles initial (before exclusion):
    subclass_name  ARandomExtend2  BRandomExtend2
    tomo_id                                      
    charlie                   300             250
    Total                     300             250
Pickled  MPS object to coloc/after_exclusion_particles/ARandomExtend2_BRandomExtend2/ARandomExtend2_BRandomExtend2.pkl
N particles after exclusion:
    subclass_name  ARandomExtend2  BRandomExtend2
    tomo_id                                   

In [42]:
# Colocalization

# colocalization name
coloc_name = 'BRandomExtend2_ARandomExtend2'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: BRandomExtend2_ARandomExtend2
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/BRandomExtend2_ARandomExtend2/BRandomExtend2_ARandomExtend2.pkl
N particles initial (before exclusion):
    subclass_name  ARandomExtend2  BRandomExtend2
    tomo_id                                      
    charlie                   300             250
    Total                     300             250
Pickled  MPS object to coloc/after_exclusion_particles/BRandomExtend2_ARandomExtend2/BRandomExtend2_ARandomExtend2.pkl
N particles after exclusion:
    subclass_name  ARandomExtend2  BRandomExtend2
    tomo_id                                   

#### Exclusion

In [43]:
# Colocalization

# colocalization name
coloc_name = 'ARandomExclFull_BRandomFull'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: ARandomExclFull_BRandomFull
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/ARandomExclFull_BRandomFull/ARandomExclFull_BRandomFull.pkl
N particles initial (before exclusion):
    subclass_name  ARandomExclFull  BRandomFull
    tomo_id                                    
    alpha                      300          250
    Total                      300          250
Pickled  MPS object to coloc/after_exclusion_particles/ARandomExclFull_BRandomFull/ARandomExclFull_BRandomFull.pkl
N particles after exclusion:
    subclass_name  ARandomExclFull  BRandomFull
    tomo_id                                    
    alpha         

In [44]:
# Colocalization

# colocalization name
coloc_name = 'BRandomFull_ARandomExclFull'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: BRandomFull_ARandomExclFull
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/BRandomFull_ARandomExclFull/BRandomFull_ARandomExclFull.pkl
N particles initial (before exclusion):
    subclass_name  ARandomExclFull  BRandomFull
    tomo_id                                    
    alpha                      300          250
    Total                      300          250
Pickled  MPS object to coloc/after_exclusion_particles/BRandomFull_ARandomExclFull/BRandomFull_ARandomExclFull.pkl
N particles after exclusion:
    subclass_name  ARandomExclFull  BRandomFull
    tomo_id                                    
    alpha         

In [45]:
# Colocalization

# colocalization name
coloc_name = 'ARandomFull_BRandomExclFull'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: ARandomFull_BRandomExclFull
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/ARandomFull_BRandomExclFull/ARandomFull_BRandomExclFull.pkl
N particles initial (before exclusion):
    subclass_name  ARandomFull  BRandomExclFull
    tomo_id                                    
    alpha                  300              250
    Total                  300              250
Pickled  MPS object to coloc/after_exclusion_particles/ARandomFull_BRandomExclFull/ARandomFull_BRandomExclFull.pkl
N particles after exclusion:
    subclass_name  ARandomFull  BRandomExclFull
    tomo_id                                    
    alpha         

### Hard clusters 08

In [45]:
# Colocalization

# colocalization name
coloc_name = 'ARandomFull_BRandom08'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: ARandomFull_BRandom08
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/ARandomFull_BRandom08/ARandomFull_BRandom08.pkl
N particles initial (before exclusion):
    subclass_name  ARandomFull  BRandom08
    tomo_id                              
    alpha                  300        250
    Total                  300        250
Pickled  MPS object to coloc/after_exclusion_particles/ARandomFull_BRandom08/ARandomFull_BRandom08.pkl
N particles after exclusion:
    subclass_name  ARandomFull  BRandom08
    tomo_id                              
    alpha                  300        250
    Total                  300        250


In [47]:
# Colocalization

# colocalization name
coloc_name = 'ARandom08_BRandomFull'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: ARandom08_BRandomFull
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/ARandom08_BRandomFull/ARandom08_BRandomFull.pkl
N particles initial (before exclusion):
    subclass_name  ARandom08  BRandomFull
    tomo_id                              
    alpha                300          250
    Total                300          250
Pickled  MPS object to coloc/after_exclusion_particles/ARandom08_BRandomFull/ARandom08_BRandomFull.pkl
N particles after exclusion:
    subclass_name  ARandom08  BRandomFull
    tomo_id                              
    alpha                300          250
    Total                300          250


In [48]:
# Colocalization

# colocalization name
coloc_name = 'BRandom08_ARandomFull'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: BRandom08_ARandomFull
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/BRandom08_ARandomFull/BRandom08_ARandomFull.pkl
N particles initial (before exclusion):
    subclass_name  ARandomFull  BRandom08
    tomo_id                              
    alpha                  300        250
    Total                  300        250
Pickled  MPS object to coloc/after_exclusion_particles/BRandom08_ARandomFull/BRandom08_ARandomFull.pkl
N particles after exclusion:
    subclass_name  ARandomFull  BRandom08
    tomo_id                              
    alpha                  300        250
    Total                  300        250


In [49]:
# Colocalization

# colocalization name
coloc_name = 'BRandomFull_ARandom08'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: BRandomFull_ARandom08
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/BRandomFull_ARandom08/BRandomFull_ARandom08.pkl
N particles initial (before exclusion):
    subclass_name  ARandom08  BRandomFull
    tomo_id                              
    alpha                300          250
    Total                300          250
Pickled  MPS object to coloc/after_exclusion_particles/BRandomFull_ARandom08/BRandomFull_ARandom08.pkl
N particles after exclusion:
    subclass_name  ARandom08  BRandomFull
    tomo_id                              
    alpha                300          250
    Total                300          250


In [50]:
# Colocalization

# colocalization name
coloc_name = 'ARandom08_BRandom08'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: ARandom08_BRandom08
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/ARandom08_BRandom08/ARandom08_BRandom08.pkl
N particles initial (before exclusion):
    subclass_name  ARandom08  BRandom08
    tomo_id                            
    alpha                300        250
    Total                300        250
Pickled  MPS object to coloc/after_exclusion_particles/ARandom08_BRandom08/ARandom08_BRandom08.pkl
N particles after exclusion:
    subclass_name  ARandom08  BRandom08
    tomo_id                            
    alpha                300        250
    Total                300        250


Processing tomo alpha ..

### Hard clusters 06, 05

#### Complete overlap

In [51]:
# Colocalization

# colocalization name
coloc_name = 'BRandom08_ARandom08'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: BRandom08_ARandom08
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/BRandom08_ARandom08/BRandom08_ARandom08.pkl
N particles initial (before exclusion):
    subclass_name  ARandom08  BRandom08
    tomo_id                            
    alpha                300        250
    Total                300        250
Pickled  MPS object to coloc/after_exclusion_particles/BRandom08_ARandom08/BRandom08_ARandom08.pkl
N particles after exclusion:
    subclass_name  ARandom08  BRandom08
    tomo_id                            
    alpha                300        250
    Total                300        250


Processing tomo alpha ..

In [52]:
# Colocalization

# colocalization name
coloc_name = 'ARandom06_BRandom06'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: ARandom06_BRandom06
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/ARandom06_BRandom06/ARandom06_BRandom06.pkl
N particles initial (before exclusion):
    subclass_name  ARandom06  BRandom06
    tomo_id                            
    alpha                300        250
    Total                300        250
Pickled  MPS object to coloc/after_exclusion_particles/ARandom06_BRandom06/ARandom06_BRandom06.pkl
N particles after exclusion:
    subclass_name  ARandom06  BRandom06
    tomo_id                            
    alpha                300        250
    Total                300        250


Processing tomo alpha ..

In [53]:
# Colocalization

# colocalization name
coloc_name = 'BRandom06_ARandom06'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: BRandom06_ARandom06
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/BRandom06_ARandom06/BRandom06_ARandom06.pkl
N particles initial (before exclusion):
    subclass_name  ARandom06  BRandom06
    tomo_id                            
    alpha                300        250
    Total                300        250
Pickled  MPS object to coloc/after_exclusion_particles/BRandom06_ARandom06/BRandom06_ARandom06.pkl
N particles after exclusion:
    subclass_name  ARandom06  BRandom06
    tomo_id                            
    alpha                300        250
    Total                300        250


Processing tomo alpha ..

In [54]:
# Colocalization

# colocalization name
coloc_name = 'ARandom05_BRandom05'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: ARandom05_BRandom05
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/ARandom05_BRandom05/ARandom05_BRandom05.pkl
N particles initial (before exclusion):
    subclass_name  ARandom05  BRandom05
    tomo_id                            
    alpha                300        250
    Total                300        250
Pickled  MPS object to coloc/after_exclusion_particles/ARandom05_BRandom05/ARandom05_BRandom05.pkl
N particles after exclusion:
    subclass_name  ARandom05  BRandom05
    tomo_id                            
    alpha                300        250
    Total                300        250


Processing tomo alpha ..

In [55]:
# Colocalization

# colocalization name
coloc_name = 'BRandom05_ARandom05'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: BRandom05_ARandom05
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/BRandom05_ARandom05/BRandom05_ARandom05.pkl
N particles initial (before exclusion):
    subclass_name  ARandom05  BRandom05
    tomo_id                            
    alpha                300        250
    Total                300        250
Pickled  MPS object to coloc/after_exclusion_particles/BRandom05_ARandom05/BRandom05_ARandom05.pkl
N particles after exclusion:
    subclass_name  ARandom05  BRandom05
    tomo_id                            
    alpha                300        250
    Total                300        250


Processing tomo alpha ..

In [56]:
# Colocalization

# colocalization name
coloc_name = 'ARandom06_BRandomFull'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: ARandom06_BRandomFull
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/ARandom06_BRandomFull/ARandom06_BRandomFull.pkl
N particles initial (before exclusion):
    subclass_name  ARandom06  BRandomFull
    tomo_id                              
    alpha                300          250
    Total                300          250
Pickled  MPS object to coloc/after_exclusion_particles/ARandom06_BRandomFull/ARandom06_BRandomFull.pkl
N particles after exclusion:
    subclass_name  ARandom06  BRandomFull
    tomo_id                              
    alpha                300          250
    Total                300          250


In [57]:
# Colocalization

# colocalization name
coloc_name = 'BRandomFull_ARandom06'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: BRandomFull_ARandom06
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/BRandomFull_ARandom06/BRandomFull_ARandom06.pkl
N particles initial (before exclusion):
    subclass_name  ARandom06  BRandomFull
    tomo_id                              
    alpha                300          250
    Total                300          250
Pickled  MPS object to coloc/after_exclusion_particles/BRandomFull_ARandom06/BRandomFull_ARandom06.pkl
N particles after exclusion:
    subclass_name  ARandom06  BRandomFull
    tomo_id                              
    alpha                300          250
    Total                300          250


In [58]:
# Colocalization

# colocalization name
coloc_name = 'ARandomFull_BRandom06'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: ARandomFull_BRandom06
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/ARandomFull_BRandom06/ARandomFull_BRandom06.pkl
N particles initial (before exclusion):
    subclass_name  ARandomFull  BRandom06
    tomo_id                              
    alpha                  300        250
    Total                  300        250
Pickled  MPS object to coloc/after_exclusion_particles/ARandomFull_BRandom06/ARandomFull_BRandom06.pkl
N particles after exclusion:
    subclass_name  ARandomFull  BRandom06
    tomo_id                              
    alpha                  300        250
    Total                  300        250


In [59]:
# Colocalization

# colocalization name
coloc_name = 'ARandom06_BRandom08'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: ARandom06_BRandom08
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/ARandom06_BRandom08/ARandom06_BRandom08.pkl
N particles initial (before exclusion):
    subclass_name  ARandom06  BRandom08
    tomo_id                            
    alpha                300        250
    Total                300        250
Pickled  MPS object to coloc/after_exclusion_particles/ARandom06_BRandom08/ARandom06_BRandom08.pkl
N particles after exclusion:
    subclass_name  ARandom06  BRandom08
    tomo_id                            
    alpha                300        250
    Total                300        250


Processing tomo alpha ..

In [60]:
# Colocalization

# colocalization name
coloc_name = 'ARandom05_BRandomFull'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: ARandom05_BRandomFull
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/ARandom05_BRandomFull/ARandom05_BRandomFull.pkl
N particles initial (before exclusion):
    subclass_name  ARandom05  BRandomFull
    tomo_id                              
    alpha                300          250
    Total                300          250
Pickled  MPS object to coloc/after_exclusion_particles/ARandom05_BRandomFull/ARandom05_BRandomFull.pkl
N particles after exclusion:
    subclass_name  ARandom05  BRandomFull
    tomo_id                              
    alpha                300          250
    Total                300          250


In [61]:
# Colocalization

# colocalization name
coloc_name = 'BRandomFull_ARandom05'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: BRandomFull_ARandom05
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/BRandomFull_ARandom05/BRandomFull_ARandom05.pkl
N particles initial (before exclusion):
    subclass_name  ARandom05  BRandomFull
    tomo_id                              
    alpha                300          250
    Total                300          250
Pickled  MPS object to coloc/after_exclusion_particles/BRandomFull_ARandom05/BRandomFull_ARandom05.pkl
N particles after exclusion:
    subclass_name  ARandom05  BRandomFull
    tomo_id                              
    alpha                300          250
    Total                300          250


In [62]:
# Colocalization

# colocalization name
coloc_name = 'ARandomFull_BRandom05'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: ARandomFull_BRandom05
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/ARandomFull_BRandom05/ARandomFull_BRandom05.pkl
N particles initial (before exclusion):
    subclass_name  ARandomFull  BRandom05
    tomo_id                              
    alpha                  300        250
    Total                  300        250
Pickled  MPS object to coloc/after_exclusion_particles/ARandomFull_BRandom05/ARandomFull_BRandom05.pkl
N particles after exclusion:
    subclass_name  ARandomFull  BRandom05
    tomo_id                              
    alpha                  300        250
    Total                  300        250


#### Partial overlap

In [63]:
# Colocalization

# colocalization name
coloc_name = 'ARandom06w_BRandom06e'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: ARandom06w_BRandom06e
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/ARandom06w_BRandom06e/ARandom06w_BRandom06e.pkl
N particles initial (before exclusion):
    subclass_name  ARandom06w  BRandom06e
    tomo_id                              
    alpha                 300         250
    Total                 300         250
Pickled  MPS object to coloc/after_exclusion_particles/ARandom06w_BRandom06e/ARandom06w_BRandom06e.pkl
N particles after exclusion:
    subclass_name  ARandom06w  BRandom06e
    tomo_id                              
    alpha                 300         250
    Total                 300         250


In [64]:
# Colocalization

# colocalization name
coloc_name = 'BRandom06e_ARandom06w'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: BRandom06e_ARandom06w
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/BRandom06e_ARandom06w/BRandom06e_ARandom06w.pkl
N particles initial (before exclusion):
    subclass_name  ARandom06w  BRandom06e
    tomo_id                              
    alpha                 300         250
    Total                 300         250
Pickled  MPS object to coloc/after_exclusion_particles/BRandom06e_ARandom06w/BRandom06e_ARandom06w.pkl
N particles after exclusion:
    subclass_name  ARandom06w  BRandom06e
    tomo_id                              
    alpha                 300         250
    Total                 300         250


In [65]:
# Colocalization

# colocalization name
coloc_name = 'ARandom06sw_BRandom06ne'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: ARandom06sw_BRandom06ne
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/ARandom06sw_BRandom06ne/ARandom06sw_BRandom06ne.pkl
N particles initial (before exclusion):
    subclass_name  ARandom06sw  BRandom06ne
    tomo_id                                
    alpha                  300          250
    Total                  300          250
Pickled  MPS object to coloc/after_exclusion_particles/ARandom06sw_BRandom06ne/ARandom06sw_BRandom06ne.pkl
N particles after exclusion:
    subclass_name  ARandom06sw  BRandom06ne
    tomo_id                                
    alpha                  300          250
    Total         

In [66]:
# Colocalization

# colocalization name
coloc_name = 'BRandom06ne_ARandom06sw'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: BRandom06ne_ARandom06sw
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/BRandom06ne_ARandom06sw/BRandom06ne_ARandom06sw.pkl
N particles initial (before exclusion):
    subclass_name  ARandom06sw  BRandom06ne
    tomo_id                                
    alpha                  300          250
    Total                  300          250
Pickled  MPS object to coloc/after_exclusion_particles/BRandom06ne_ARandom06sw/BRandom06ne_ARandom06sw.pkl
N particles after exclusion:
    subclass_name  ARandom06sw  BRandom06ne
    tomo_id                                
    alpha                  300          250
    Total         

In [67]:
# Colocalization

# colocalization name
coloc_name = 'ARandom05w_BRandom05e'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: ARandom05w_BRandom05e
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/ARandom05w_BRandom05e/ARandom05w_BRandom05e.pkl
N particles initial (before exclusion):
    subclass_name  ARandom05w  BRandom05e
    tomo_id                              
    alpha                 300         250
    Total                 300         250
Pickled  MPS object to coloc/after_exclusion_particles/ARandom05w_BRandom05e/ARandom05w_BRandom05e.pkl
N particles after exclusion:
    subclass_name  ARandom05w  BRandom05e
    tomo_id                              
    alpha                 300         250
    Total                 300         250


In [68]:
# Colocalization

# colocalization name
coloc_name = 'BRandom05e_ARandom05w'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: BRandom05e_ARandom05w
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/BRandom05e_ARandom05w/BRandom05e_ARandom05w.pkl
N particles initial (before exclusion):
    subclass_name  ARandom05w  BRandom05e
    tomo_id                              
    alpha                 300         250
    Total                 300         250
Pickled  MPS object to coloc/after_exclusion_particles/BRandom05e_ARandom05w/BRandom05e_ARandom05w.pkl
N particles after exclusion:
    subclass_name  ARandom05w  BRandom05e
    tomo_id                              
    alpha                 300         250
    Total                 300         250


In [69]:
# Colocalization

# colocalization name
coloc_name = 'ARandom05sw_BRandom05ne'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: ARandom05sw_BRandom05ne
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/ARandom05sw_BRandom05ne/ARandom05sw_BRandom05ne.pkl
N particles initial (before exclusion):
    subclass_name  ARandom05sw  BRandom05ne
    tomo_id                                
    alpha                  300          250
    Total                  300          250
Pickled  MPS object to coloc/after_exclusion_particles/ARandom05sw_BRandom05ne/ARandom05sw_BRandom05ne.pkl
N particles after exclusion:
    subclass_name  ARandom05sw  BRandom05ne
    tomo_id                                
    alpha                  300          250
    Total         

In [70]:
# Colocalization

# colocalization name
coloc_name = 'BRandom05ne_ARandom05sw'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: BRandom05ne_ARandom05sw
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/BRandom05ne_ARandom05sw/BRandom05ne_ARandom05sw.pkl
N particles initial (before exclusion):
    subclass_name  ARandom05sw  BRandom05ne
    tomo_id                                
    alpha                  300          250
    Total                  300          250
Pickled  MPS object to coloc/after_exclusion_particles/BRandom05ne_ARandom05sw/BRandom05ne_ARandom05sw.pkl
N particles after exclusion:
    subclass_name  ARandom05sw  BRandom05ne
    tomo_id                                
    alpha                  300          250
    Total         

### Hard clusters A-C

#### Full

In [71]:
# Colocalization

# colocalization name
coloc_name = 'ARandomFull_CRandomFull'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: ARandomFull_CRandomFull
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/ARandomFull_CRandomFull/ARandomFull_CRandomFull.pkl
N particles initial (before exclusion):
    subclass_name  ARandomFull  CRandomFull
    tomo_id                                
    alpha                  300           50
    Total                  300           50
Pickled  MPS object to coloc/after_exclusion_particles/ARandomFull_CRandomFull/ARandomFull_CRandomFull.pkl
N particles after exclusion:
    subclass_name  ARandomFull  CRandomFull
    tomo_id                                
    alpha                  300           50
    Total         

In [72]:
# Colocalization

# colocalization name
coloc_name = 'CRandomFull_ARandomFull'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: CRandomFull_ARandomFull
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/CRandomFull_ARandomFull/CRandomFull_ARandomFull.pkl
N particles initial (before exclusion):
    subclass_name  ARandomFull  CRandomFull
    tomo_id                                
    alpha                  300           50
    Total                  300           50
Pickled  MPS object to coloc/after_exclusion_particles/CRandomFull_ARandomFull/CRandomFull_ARandomFull.pkl
N particles after exclusion:
    subclass_name  ARandomFull  CRandomFull
    tomo_id                                
    alpha                  300           50
    Total         

#### Complete overlap

In [73]:
# Colocalization

# colocalization name
coloc_name = 'ARandom08_CRandom08'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: ARandom08_CRandom08
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/ARandom08_CRandom08/ARandom08_CRandom08.pkl
N particles initial (before exclusion):
    subclass_name  ARandom08  CRandom08
    tomo_id                            
    alpha                300         50
    Total                300         50
Pickled  MPS object to coloc/after_exclusion_particles/ARandom08_CRandom08/ARandom08_CRandom08.pkl
N particles after exclusion:
    subclass_name  ARandom08  CRandom08
    tomo_id                            
    alpha                300         50
    Total                300         50


Processing tomo alpha ..

In [74]:
# Colocalization

# colocalization name
coloc_name = 'CRandom08_ARandom08'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: CRandom08_ARandom08
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/CRandom08_ARandom08/CRandom08_ARandom08.pkl
N particles initial (before exclusion):
    subclass_name  ARandom08  CRandom08
    tomo_id                            
    alpha                300         50
    Total                300         50
Pickled  MPS object to coloc/after_exclusion_particles/CRandom08_ARandom08/CRandom08_ARandom08.pkl
N particles after exclusion:
    subclass_name  ARandom08  CRandom08
    tomo_id                            
    alpha                300         50
    Total                300         50


Processing tomo alpha ..

In [75]:
# Colocalization

# colocalization name
coloc_name = 'ARandom06_CRandom06'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: ARandom06_CRandom06
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/ARandom06_CRandom06/ARandom06_CRandom06.pkl
N particles initial (before exclusion):
    subclass_name  ARandom06  CRandom06
    tomo_id                            
    alpha                300         50
    Total                300         50
Pickled  MPS object to coloc/after_exclusion_particles/ARandom06_CRandom06/ARandom06_CRandom06.pkl
N particles after exclusion:
    subclass_name  ARandom06  CRandom06
    tomo_id                            
    alpha                300         50
    Total                300         50


Processing tomo alpha ..

In [76]:
# Colocalization

# colocalization name
coloc_name = 'CRandom06_ARandom06'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: CRandom06_ARandom06
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/CRandom06_ARandom06/CRandom06_ARandom06.pkl
N particles initial (before exclusion):
    subclass_name  ARandom06  CRandom06
    tomo_id                            
    alpha                300         50
    Total                300         50
Pickled  MPS object to coloc/after_exclusion_particles/CRandom06_ARandom06/CRandom06_ARandom06.pkl
N particles after exclusion:
    subclass_name  ARandom06  CRandom06
    tomo_id                            
    alpha                300         50
    Total                300         50


Processing tomo alpha ..

In [77]:
# Colocalization

# colocalization name
coloc_name = 'ARandom05_CRandom05'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: ARandom05_CRandom05
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/ARandom05_CRandom05/ARandom05_CRandom05.pkl
N particles initial (before exclusion):
    subclass_name  ARandom05  CRandom05
    tomo_id                            
    alpha                300         50
    Total                300         50
Pickled  MPS object to coloc/after_exclusion_particles/ARandom05_CRandom05/ARandom05_CRandom05.pkl
N particles after exclusion:
    subclass_name  ARandom05  CRandom05
    tomo_id                            
    alpha                300         50
    Total                300         50


Processing tomo alpha ..

In [78]:
# Colocalization

# colocalization name
coloc_name = 'CRandom05_ARandom05'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: CRandom05_ARandom05
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/CRandom05_ARandom05/CRandom05_ARandom05.pkl
N particles initial (before exclusion):
    subclass_name  ARandom05  CRandom05
    tomo_id                            
    alpha                300         50
    Total                300         50
Pickled  MPS object to coloc/after_exclusion_particles/CRandom05_ARandom05/CRandom05_ARandom05.pkl
N particles after exclusion:
    subclass_name  ARandom05  CRandom05
    tomo_id                            
    alpha                300         50
    Total                300         50


Processing tomo alpha ..

#### With full, 08

In [79]:
# Colocalization

# colocalization name
coloc_name = 'ARandom06_CRandomFull'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: ARandom06_CRandomFull
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/ARandom06_CRandomFull/ARandom06_CRandomFull.pkl
N particles initial (before exclusion):
    subclass_name  ARandom06  CRandomFull
    tomo_id                              
    alpha                300           50
    Total                300           50
Pickled  MPS object to coloc/after_exclusion_particles/ARandom06_CRandomFull/ARandom06_CRandomFull.pkl
N particles after exclusion:
    subclass_name  ARandom06  CRandomFull
    tomo_id                              
    alpha                300           50
    Total                300           50


In [80]:
# Colocalization

# colocalization name
coloc_name = 'CRandomFull_ARandom06'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: CRandomFull_ARandom06
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/CRandomFull_ARandom06/CRandomFull_ARandom06.pkl
N particles initial (before exclusion):
    subclass_name  ARandom06  CRandomFull
    tomo_id                              
    alpha                300           50
    Total                300           50
Pickled  MPS object to coloc/after_exclusion_particles/CRandomFull_ARandom06/CRandomFull_ARandom06.pkl
N particles after exclusion:
    subclass_name  ARandom06  CRandomFull
    tomo_id                              
    alpha                300           50
    Total                300           50


In [81]:
# Colocalization

# colocalization name
coloc_name = 'ARandomFull_CRandom06'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: ARandomFull_CRandom06
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/ARandomFull_CRandom06/ARandomFull_CRandom06.pkl
N particles initial (before exclusion):
    subclass_name  ARandomFull  CRandom06
    tomo_id                              
    alpha                  300         50
    Total                  300         50
Pickled  MPS object to coloc/after_exclusion_particles/ARandomFull_CRandom06/ARandomFull_CRandom06.pkl
N particles after exclusion:
    subclass_name  ARandomFull  CRandom06
    tomo_id                              
    alpha                  300         50
    Total                  300         50


In [82]:
# Colocalization

# colocalization name
coloc_name = 'CRandom06_ARandomFull'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: CRandom06_ARandomFull
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/CRandom06_ARandomFull/CRandom06_ARandomFull.pkl
N particles initial (before exclusion):
    subclass_name  ARandomFull  CRandom06
    tomo_id                              
    alpha                  300         50
    Total                  300         50
Pickled  MPS object to coloc/after_exclusion_particles/CRandom06_ARandomFull/CRandom06_ARandomFull.pkl
N particles after exclusion:
    subclass_name  ARandomFull  CRandom06
    tomo_id                              
    alpha                  300         50
    Total                  300         50


### Interaction mode 3 A-B

#### Size full and 08

In [83]:
# colocalization name
coloc_name = 'ARandomFull_BrfCoclustArfp1-p1-d10'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: ARandomFull_BrfCoclustArfp1-p1-d10
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/ARandomFull_BrfCoclustArfp1-p1-d10/ARandomFull_BrfCoclustArfp1-p1-d10.pkl
N particles initial (before exclusion):
    subclass_name  ARandomFull  BrfCoclustArfp1-p1-d10
    tomo_id                                           
    alpha                  300                     250
    Total                  300                     250
Pickled  MPS object to coloc/after_exclusion_particles/ARandomFull_BrfCoclustArfp1-p1-d10/ARandomFull_BrfCoclustArfp1-p1-d10.pkl
N particles after exclusion:
    subclass_name  ARandomFull  BrfCoclustArfp1-p1-

In [84]:
# colocalization name
coloc_name = 'BrfCoclustArfp1-p1-d10_ARandomFull'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: BrfCoclustArfp1-p1-d10_ARandomFull
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/BrfCoclustArfp1-p1-d10_ARandomFull/BrfCoclustArfp1-p1-d10_ARandomFull.pkl
N particles initial (before exclusion):
    subclass_name  ARandomFull  BrfCoclustArfp1-p1-d10
    tomo_id                                           
    alpha                  300                     250
    Total                  300                     250
Pickled  MPS object to coloc/after_exclusion_particles/BrfCoclustArfp1-p1-d10_ARandomFull/BrfCoclustArfp1-p1-d10_ARandomFull.pkl
N particles after exclusion:
    subclass_name  ARandomFull  BrfCoclustArfp1-p1-

In [85]:
# colocalization name
coloc_name = 'ARandomFull_BrfCoclustArfp07-p07-d10'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: ARandomFull_BrfCoclustArfp07-p07-d10
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/ARandomFull_BrfCoclustArfp07-p07-d10/ARandomFull_BrfCoclustArfp07-p07-d10.pkl
N particles initial (before exclusion):
    subclass_name  ARandomFull  BrfCoclustArfp07-p07-d10
    tomo_id                                             
    alpha                  300                       250
    Total                  300                       250
Pickled  MPS object to coloc/after_exclusion_particles/ARandomFull_BrfCoclustArfp07-p07-d10/ARandomFull_BrfCoclustArfp07-p07-d10.pkl
N particles after exclusion:
    subclass_name  ARandomFull  B

In [86]:
# colocalization name
coloc_name = 'BrfCoclustArfp07-p07-d10_ARandomFull'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: BrfCoclustArfp07-p07-d10_ARandomFull
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/BrfCoclustArfp07-p07-d10_ARandomFull/BrfCoclustArfp07-p07-d10_ARandomFull.pkl
N particles initial (before exclusion):
    subclass_name  ARandomFull  BrfCoclustArfp07-p07-d10
    tomo_id                                             
    alpha                  300                       250
    Total                  300                       250
Pickled  MPS object to coloc/after_exclusion_particles/BrfCoclustArfp07-p07-d10_ARandomFull/BrfCoclustArfp07-p07-d10_ARandomFull.pkl
N particles after exclusion:
    subclass_name  ARandomFull  B

In [87]:
# colocalization name
coloc_name = 'ARandomFull_BrfCoclustArfp06-p06-d10'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: ARandomFull_BrfCoclustArfp06-p06-d10
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/ARandomFull_BrfCoclustArfp06-p06-d10/ARandomFull_BrfCoclustArfp06-p06-d10.pkl
N particles initial (before exclusion):
    subclass_name  ARandomFull  BrfCoclustArfp06-p06-d10
    tomo_id                                             
    alpha                  300                       250
    Total                  300                       250
Pickled  MPS object to coloc/after_exclusion_particles/ARandomFull_BrfCoclustArfp06-p06-d10/ARandomFull_BrfCoclustArfp06-p06-d10.pkl
N particles after exclusion:
    subclass_name  ARandomFull  B

In [88]:
# colocalization name
coloc_name = 'BrfCoclustArfp06-p06-d10_ARandomFull'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: BrfCoclustArfp06-p06-d10_ARandomFull
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/BrfCoclustArfp06-p06-d10_ARandomFull/BrfCoclustArfp06-p06-d10_ARandomFull.pkl
N particles initial (before exclusion):
    subclass_name  ARandomFull  BrfCoclustArfp06-p06-d10
    tomo_id                                             
    alpha                  300                       250
    Total                  300                       250
Pickled  MPS object to coloc/after_exclusion_particles/BrfCoclustArfp06-p06-d10_ARandomFull/BrfCoclustArfp06-p06-d10_ARandomFull.pkl
N particles after exclusion:
    subclass_name  ARandomFull  B

In [89]:
# colocalization name
coloc_name = 'ARandomFull_BrfCoclustArfp05-p05-d10'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: ARandomFull_BrfCoclustArfp05-p05-d10
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/ARandomFull_BrfCoclustArfp05-p05-d10/ARandomFull_BrfCoclustArfp05-p05-d10.pkl
N particles initial (before exclusion):
    subclass_name  ARandomFull  BrfCoclustArfp05-p05-d10
    tomo_id                                             
    alpha                  300                       250
    Total                  300                       250
Pickled  MPS object to coloc/after_exclusion_particles/ARandomFull_BrfCoclustArfp05-p05-d10/ARandomFull_BrfCoclustArfp05-p05-d10.pkl
N particles after exclusion:
    subclass_name  ARandomFull  B

In [90]:
# colocalization name
coloc_name = 'BrfCoclustArfp05-p05-d10_ARandomFull'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: BrfCoclustArfp05-p05-d10_ARandomFull
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/BrfCoclustArfp05-p05-d10_ARandomFull/BrfCoclustArfp05-p05-d10_ARandomFull.pkl
N particles initial (before exclusion):
    subclass_name  ARandomFull  BrfCoclustArfp05-p05-d10
    tomo_id                                             
    alpha                  300                       250
    Total                  300                       250
Pickled  MPS object to coloc/after_exclusion_particles/BrfCoclustArfp05-p05-d10_ARandomFull/BrfCoclustArfp05-p05-d10_ARandomFull.pkl
N particles after exclusion:
    subclass_name  ARandomFull  B

In [91]:
# colocalization name
coloc_name = 'ARandomFull_BrfCoclustArfp04-p04-d10'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: ARandomFull_BrfCoclustArfp04-p04-d10
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/ARandomFull_BrfCoclustArfp04-p04-d10/ARandomFull_BrfCoclustArfp04-p04-d10.pkl
N particles initial (before exclusion):
    subclass_name  ARandomFull  BrfCoclustArfp04-p04-d10
    tomo_id                                             
    alpha                  300                       250
    Total                  300                       250
Pickled  MPS object to coloc/after_exclusion_particles/ARandomFull_BrfCoclustArfp04-p04-d10/ARandomFull_BrfCoclustArfp04-p04-d10.pkl
N particles after exclusion:
    subclass_name  ARandomFull  B

In [92]:
# colocalization name
coloc_name = 'BrfCoclustArfp04-p04-d10_ARandomFull'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: BrfCoclustArfp04-p04-d10_ARandomFull
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/BrfCoclustArfp04-p04-d10_ARandomFull/BrfCoclustArfp04-p04-d10_ARandomFull.pkl
N particles initial (before exclusion):
    subclass_name  ARandomFull  BrfCoclustArfp04-p04-d10
    tomo_id                                             
    alpha                  300                       250
    Total                  300                       250
Pickled  MPS object to coloc/after_exclusion_particles/BrfCoclustArfp04-p04-d10_ARandomFull/BrfCoclustArfp04-p04-d10_ARandomFull.pkl
N particles after exclusion:
    subclass_name  ARandomFull  B

In [93]:
# colocalization name
coloc_name = 'ARandomFull_BrfCoclustArfp03-p03-d10'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: ARandomFull_BrfCoclustArfp03-p03-d10
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/ARandomFull_BrfCoclustArfp03-p03-d10/ARandomFull_BrfCoclustArfp03-p03-d10.pkl
N particles initial (before exclusion):
    subclass_name  ARandomFull  BrfCoclustArfp03-p03-d10
    tomo_id                                             
    alpha                  300                       250
    Total                  300                       250
Pickled  MPS object to coloc/after_exclusion_particles/ARandomFull_BrfCoclustArfp03-p03-d10/ARandomFull_BrfCoclustArfp03-p03-d10.pkl
N particles after exclusion:
    subclass_name  ARandomFull  B

In [94]:
# colocalization name
coloc_name = 'BrfCoclustArfp03-p03-d10_ARandomFull'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: BrfCoclustArfp03-p03-d10_ARandomFull
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/BrfCoclustArfp03-p03-d10_ARandomFull/BrfCoclustArfp03-p03-d10_ARandomFull.pkl
N particles initial (before exclusion):
    subclass_name  ARandomFull  BrfCoclustArfp03-p03-d10
    tomo_id                                             
    alpha                  300                       250
    Total                  300                       250
Pickled  MPS object to coloc/after_exclusion_particles/BrfCoclustArfp03-p03-d10_ARandomFull/BrfCoclustArfp03-p03-d10_ARandomFull.pkl
N particles after exclusion:
    subclass_name  ARandomFull  B

#### Size full 1-to-1

In [95]:
# colocalization name
coloc_name = 'ARandomFull_BrfCoclustArfp07-pkd-d10'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: ARandomFull_BrfCoclustArfp07-pkd-d10
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/ARandomFull_BrfCoclustArfp07-pkd-d10/ARandomFull_BrfCoclustArfp07-pkd-d10.pkl
N particles initial (before exclusion):
    subclass_name  ARandomFull  BrfCoclustArfp07-pkd-d10
    tomo_id                                             
    alpha                  300                       250
    Total                  300                       250
Pickled  MPS object to coloc/after_exclusion_particles/ARandomFull_BrfCoclustArfp07-pkd-d10/ARandomFull_BrfCoclustArfp07-pkd-d10.pkl
N particles after exclusion:
    subclass_name  ARandomFull  B

In [96]:
# colocalization name
coloc_name = 'BrfCoclustArfp07-pkd-d10_ARandomFull'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: BrfCoclustArfp07-pkd-d10_ARandomFull
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/BrfCoclustArfp07-pkd-d10_ARandomFull/BrfCoclustArfp07-pkd-d10_ARandomFull.pkl
N particles initial (before exclusion):
    subclass_name  ARandomFull  BrfCoclustArfp07-pkd-d10
    tomo_id                                             
    alpha                  300                       250
    Total                  300                       250
Pickled  MPS object to coloc/after_exclusion_particles/BrfCoclustArfp07-pkd-d10_ARandomFull/BrfCoclustArfp07-pkd-d10_ARandomFull.pkl
N particles after exclusion:
    subclass_name  ARandomFull  B

In [97]:
# colocalization name
coloc_name = 'ARandomFull_BrfCoclustArfp03-pkd-d10'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: ARandomFull_BrfCoclustArfp03-pkd-d10
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/ARandomFull_BrfCoclustArfp03-pkd-d10/ARandomFull_BrfCoclustArfp03-pkd-d10.pkl
N particles initial (before exclusion):
    subclass_name  ARandomFull  BrfCoclustArfp03-pkd-d10
    tomo_id                                             
    alpha                  300                       250
    Total                  300                       250
Pickled  MPS object to coloc/after_exclusion_particles/ARandomFull_BrfCoclustArfp03-pkd-d10/ARandomFull_BrfCoclustArfp03-pkd-d10.pkl
N particles after exclusion:
    subclass_name  ARandomFull  B

In [98]:
# colocalization name
coloc_name = 'BrfCoclustArfp03-pkd-d10_ARandomFull'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: BrfCoclustArfp03-pkd-d10_ARandomFull
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/BrfCoclustArfp03-pkd-d10_ARandomFull/BrfCoclustArfp03-pkd-d10_ARandomFull.pkl
N particles initial (before exclusion):
    subclass_name  ARandomFull  BrfCoclustArfp03-pkd-d10
    tomo_id                                             
    alpha                  300                       250
    Total                  300                       250
Pickled  MPS object to coloc/after_exclusion_particles/BrfCoclustArfp03-pkd-d10_ARandomFull/BrfCoclustArfp03-pkd-d10_ARandomFull.pkl
N particles after exclusion:
    subclass_name  ARandomFull  B

In [99]:
# colocalization name
coloc_name = 'ARandomFull_BrfCoclustArfp02-pkd-d10'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: ARandomFull_BrfCoclustArfp02-pkd-d10
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/ARandomFull_BrfCoclustArfp02-pkd-d10/ARandomFull_BrfCoclustArfp02-pkd-d10.pkl
N particles initial (before exclusion):
    subclass_name  ARandomFull  BrfCoclustArfp02-pkd-d10
    tomo_id                                             
    alpha                  300                       250
    Total                  300                       250
Pickled  MPS object to coloc/after_exclusion_particles/ARandomFull_BrfCoclustArfp02-pkd-d10/ARandomFull_BrfCoclustArfp02-pkd-d10.pkl
N particles after exclusion:
    subclass_name  ARandomFull  B

In [100]:
# colocalization name
coloc_name = 'BrfCoclustArfp02-pkd-d10_ARandomFull'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: BrfCoclustArfp02-pkd-d10_ARandomFull
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/BrfCoclustArfp02-pkd-d10_ARandomFull/BrfCoclustArfp02-pkd-d10_ARandomFull.pkl
N particles initial (before exclusion):
    subclass_name  ARandomFull  BrfCoclustArfp02-pkd-d10
    tomo_id                                             
    alpha                  300                       250
    Total                  300                       250
Pickled  MPS object to coloc/after_exclusion_particles/BrfCoclustArfp02-pkd-d10_ARandomFull/BrfCoclustArfp02-pkd-d10_ARandomFull.pkl
N particles after exclusion:
    subclass_name  ARandomFull  B

In [101]:
# colocalization name
coloc_name = 'ARandomFull_BrfCoclustArfp015-pkd-d10'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: ARandomFull_BrfCoclustArfp015-pkd-d10
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/ARandomFull_BrfCoclustArfp015-pkd-d10/ARandomFull_BrfCoclustArfp015-pkd-d10.pkl
N particles initial (before exclusion):
    subclass_name  ARandomFull  BrfCoclustArfp015-pkd-d10
    tomo_id                                              
    alpha                  300                        250
    Total                  300                        250
Pickled  MPS object to coloc/after_exclusion_particles/ARandomFull_BrfCoclustArfp015-pkd-d10/ARandomFull_BrfCoclustArfp015-pkd-d10.pkl
N particles after exclusion:
    subclass_name  ARand

In [102]:
# colocalization name
coloc_name = 'BrfCoclustArfp015-pkd-d10_ARandomFull'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: BrfCoclustArfp015-pkd-d10_ARandomFull
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/BrfCoclustArfp015-pkd-d10_ARandomFull/BrfCoclustArfp015-pkd-d10_ARandomFull.pkl
N particles initial (before exclusion):
    subclass_name  ARandomFull  BrfCoclustArfp015-pkd-d10
    tomo_id                                              
    alpha                  300                        250
    Total                  300                        250
Pickled  MPS object to coloc/after_exclusion_particles/BrfCoclustArfp015-pkd-d10_ARandomFull/BrfCoclustArfp015-pkd-d10_ARandomFull.pkl
N particles after exclusion:
    subclass_name  ARand

In [103]:
# colocalization name
coloc_name = 'ARandomFull_BrfCoclustArfp01-pkd-d10'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: ARandomFull_BrfCoclustArfp01-pkd-d10
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/ARandomFull_BrfCoclustArfp01-pkd-d10/ARandomFull_BrfCoclustArfp01-pkd-d10.pkl
N particles initial (before exclusion):
    subclass_name  ARandomFull  BrfCoclustArfp01-pkd-d10
    tomo_id                                             
    alpha                  300                       250
    Total                  300                       250
Pickled  MPS object to coloc/after_exclusion_particles/ARandomFull_BrfCoclustArfp01-pkd-d10/ARandomFull_BrfCoclustArfp01-pkd-d10.pkl
N particles after exclusion:
    subclass_name  ARandomFull  B

In [104]:
# colocalization name
coloc_name = 'BrfCoclustArfp005-pkd-d10_ARandomFull'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: BrfCoclustArfp005-pkd-d10_ARandomFull
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/BrfCoclustArfp005-pkd-d10_ARandomFull/BrfCoclustArfp005-pkd-d10_ARandomFull.pkl
N particles initial (before exclusion):
    subclass_name  ARandomFull  BrfCoclustArfp005-pkd-d10
    tomo_id                                              
    alpha                  300                        250
    Total                  300                        250
Pickled  MPS object to coloc/after_exclusion_particles/BrfCoclustArfp005-pkd-d10_ARandomFull/BrfCoclustArfp005-pkd-d10_ARandomFull.pkl
N particles after exclusion:
    subclass_name  ARand

In [105]:
# colocalization name
coloc_name = 'ARandomFull_BrfCoclustArfp005-pkd-d10'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: ARandomFull_BrfCoclustArfp005-pkd-d10
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/ARandomFull_BrfCoclustArfp005-pkd-d10/ARandomFull_BrfCoclustArfp005-pkd-d10.pkl
N particles initial (before exclusion):
    subclass_name  ARandomFull  BrfCoclustArfp005-pkd-d10
    tomo_id                                              
    alpha                  300                        250
    Total                  300                        250
Pickled  MPS object to coloc/after_exclusion_particles/ARandomFull_BrfCoclustArfp005-pkd-d10/ARandomFull_BrfCoclustArfp005-pkd-d10.pkl
N particles after exclusion:
    subclass_name  ARand

#### Size full and 08 1-to-1, d=5

In [106]:
# colocalization name
coloc_name = 'ARandomFull_BrfCoclustArfp07-pkd-d5'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: ARandomFull_BrfCoclustArfp07-pkd-d5
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/ARandomFull_BrfCoclustArfp07-pkd-d5/ARandomFull_BrfCoclustArfp07-pkd-d5.pkl
N particles initial (before exclusion):
    subclass_name  ARandomFull  BrfCoclustArfp07-pkd-d5
    tomo_id                                            
    alpha                  300                      250
    Total                  300                      250
Pickled  MPS object to coloc/after_exclusion_particles/ARandomFull_BrfCoclustArfp07-pkd-d5/ARandomFull_BrfCoclustArfp07-pkd-d5.pkl
N particles after exclusion:
    subclass_name  ARandomFull  BrfCoclust

In [107]:
# colocalization name
coloc_name = 'BrfCoclustArfp07-pkd-d5_ARandomFull'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: BrfCoclustArfp07-pkd-d5_ARandomFull
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/BrfCoclustArfp07-pkd-d5_ARandomFull/BrfCoclustArfp07-pkd-d5_ARandomFull.pkl
N particles initial (before exclusion):
    subclass_name  ARandomFull  BrfCoclustArfp07-pkd-d5
    tomo_id                                            
    alpha                  300                      250
    Total                  300                      250
Pickled  MPS object to coloc/after_exclusion_particles/BrfCoclustArfp07-pkd-d5_ARandomFull/BrfCoclustArfp07-pkd-d5_ARandomFull.pkl
N particles after exclusion:
    subclass_name  ARandomFull  BrfCoclust

In [108]:
# colocalization name
coloc_name = 'ARandomFull_BrfCoclustArfp03-pkd-d5'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: ARandomFull_BrfCoclustArfp03-pkd-d5
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/ARandomFull_BrfCoclustArfp03-pkd-d5/ARandomFull_BrfCoclustArfp03-pkd-d5.pkl
N particles initial (before exclusion):
    subclass_name  ARandomFull  BrfCoclustArfp03-pkd-d5
    tomo_id                                            
    alpha                  300                      250
    Total                  300                      250
Pickled  MPS object to coloc/after_exclusion_particles/ARandomFull_BrfCoclustArfp03-pkd-d5/ARandomFull_BrfCoclustArfp03-pkd-d5.pkl
N particles after exclusion:
    subclass_name  ARandomFull  BrfCoclust

In [109]:
# colocalization name
coloc_name = 'BrfCoclustArfp03-pkd-d5_ARandomFull'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: BrfCoclustArfp03-pkd-d5_ARandomFull
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/BrfCoclustArfp03-pkd-d5_ARandomFull/BrfCoclustArfp03-pkd-d5_ARandomFull.pkl
N particles initial (before exclusion):
    subclass_name  ARandomFull  BrfCoclustArfp03-pkd-d5
    tomo_id                                            
    alpha                  300                      250
    Total                  300                      250
Pickled  MPS object to coloc/after_exclusion_particles/BrfCoclustArfp03-pkd-d5_ARandomFull/BrfCoclustArfp03-pkd-d5_ARandomFull.pkl
N particles after exclusion:
    subclass_name  ARandomFull  BrfCoclust

In [110]:
# colocalization name
coloc_name = 'ARandomFull_BrfCoclustArfp02-pkd-d5'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: ARandomFull_BrfCoclustArfp02-pkd-d5
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/ARandomFull_BrfCoclustArfp02-pkd-d5/ARandomFull_BrfCoclustArfp02-pkd-d5.pkl
N particles initial (before exclusion):
    subclass_name  ARandomFull  BrfCoclustArfp02-pkd-d5
    tomo_id                                            
    alpha                  300                      250
    Total                  300                      250
Pickled  MPS object to coloc/after_exclusion_particles/ARandomFull_BrfCoclustArfp02-pkd-d5/ARandomFull_BrfCoclustArfp02-pkd-d5.pkl
N particles after exclusion:
    subclass_name  ARandomFull  BrfCoclust

In [111]:
# colocalization name
coloc_name = 'BrfCoclustArfp02-pkd-d5_ARandomFull'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: BrfCoclustArfp02-pkd-d5_ARandomFull
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/BrfCoclustArfp02-pkd-d5_ARandomFull/BrfCoclustArfp02-pkd-d5_ARandomFull.pkl
N particles initial (before exclusion):
    subclass_name  ARandomFull  BrfCoclustArfp02-pkd-d5
    tomo_id                                            
    alpha                  300                      250
    Total                  300                      250
Pickled  MPS object to coloc/after_exclusion_particles/BrfCoclustArfp02-pkd-d5_ARandomFull/BrfCoclustArfp02-pkd-d5_ARandomFull.pkl
N particles after exclusion:
    subclass_name  ARandomFull  BrfCoclust

In [112]:
# colocalization name
coloc_name = 'ARandomFull_BrfCoclustArfp015-pkd-d5'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: ARandomFull_BrfCoclustArfp015-pkd-d5
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/ARandomFull_BrfCoclustArfp015-pkd-d5/ARandomFull_BrfCoclustArfp015-pkd-d5.pkl
N particles initial (before exclusion):
    subclass_name  ARandomFull  BrfCoclustArfp015-pkd-d5
    tomo_id                                             
    alpha                  300                       250
    Total                  300                       250
Pickled  MPS object to coloc/after_exclusion_particles/ARandomFull_BrfCoclustArfp015-pkd-d5/ARandomFull_BrfCoclustArfp015-pkd-d5.pkl
N particles after exclusion:
    subclass_name  ARandomFull  B

In [113]:
# colocalization name
coloc_name = 'BrfCoclustArfp015-pkd-d5_ARandomFull'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: BrfCoclustArfp015-pkd-d5_ARandomFull
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/BrfCoclustArfp015-pkd-d5_ARandomFull/BrfCoclustArfp015-pkd-d5_ARandomFull.pkl
N particles initial (before exclusion):
    subclass_name  ARandomFull  BrfCoclustArfp015-pkd-d5
    tomo_id                                             
    alpha                  300                       250
    Total                  300                       250
Pickled  MPS object to coloc/after_exclusion_particles/BrfCoclustArfp015-pkd-d5_ARandomFull/BrfCoclustArfp015-pkd-d5_ARandomFull.pkl
N particles after exclusion:
    subclass_name  ARandomFull  B

In [114]:
# colocalization name
coloc_name = 'ARandomFull_BrfCoclustArfp01-pkd-d5'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: ARandomFull_BrfCoclustArfp01-pkd-d5
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/ARandomFull_BrfCoclustArfp01-pkd-d5/ARandomFull_BrfCoclustArfp01-pkd-d5.pkl
N particles initial (before exclusion):
    subclass_name  ARandomFull  BrfCoclustArfp01-pkd-d5
    tomo_id                                            
    alpha                  300                      250
    Total                  300                      250
Pickled  MPS object to coloc/after_exclusion_particles/ARandomFull_BrfCoclustArfp01-pkd-d5/ARandomFull_BrfCoclustArfp01-pkd-d5.pkl
N particles after exclusion:
    subclass_name  ARandomFull  BrfCoclust

In [115]:
# colocalization name
coloc_name = 'BrfCoclustArfp01-pkd-d5_ARandomFull'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: BrfCoclustArfp01-pkd-d5_ARandomFull
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/BrfCoclustArfp01-pkd-d5_ARandomFull/BrfCoclustArfp01-pkd-d5_ARandomFull.pkl
N particles initial (before exclusion):
    subclass_name  ARandomFull  BrfCoclustArfp01-pkd-d5
    tomo_id                                            
    alpha                  300                      250
    Total                  300                      250
Pickled  MPS object to coloc/after_exclusion_particles/BrfCoclustArfp01-pkd-d5_ARandomFull/BrfCoclustArfp01-pkd-d5_ARandomFull.pkl
N particles after exclusion:
    subclass_name  ARandomFull  BrfCoclust

#### Size full 1-to-1, d=15

In [116]:
# colocalization name
coloc_name = 'ARandomFull_BrfCoclustArfp03-pkd-d15'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: ARandomFull_BrfCoclustArfp03-pkd-d15
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/ARandomFull_BrfCoclustArfp03-pkd-d15/ARandomFull_BrfCoclustArfp03-pkd-d15.pkl
N particles initial (before exclusion):
    subclass_name  ARandomFull  BrfCoclustArfp03-pkd-d15
    tomo_id                                             
    alpha                  300                       250
    Total                  300                       250
Pickled  MPS object to coloc/after_exclusion_particles/ARandomFull_BrfCoclustArfp03-pkd-d15/ARandomFull_BrfCoclustArfp03-pkd-d15.pkl
N particles after exclusion:
    subclass_name  ARandomFull  B

In [117]:
# colocalization name
coloc_name = 'BrfCoclustArfp03-pkd-d15_ARandomFull'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: BrfCoclustArfp03-pkd-d15_ARandomFull
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/BrfCoclustArfp03-pkd-d15_ARandomFull/BrfCoclustArfp03-pkd-d15_ARandomFull.pkl
N particles initial (before exclusion):
    subclass_name  ARandomFull  BrfCoclustArfp03-pkd-d15
    tomo_id                                             
    alpha                  300                       250
    Total                  300                       250
Pickled  MPS object to coloc/after_exclusion_particles/BrfCoclustArfp03-pkd-d15_ARandomFull/BrfCoclustArfp03-pkd-d15_ARandomFull.pkl
N particles after exclusion:
    subclass_name  ARandomFull  B

#### Size 0.6, full overlap

In [118]:
# colocalization name
coloc_name = 'ARandom06_Br06CoclustAr06p1-p1-d10'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: ARandom06_Br06CoclustAr06p1-p1-d10
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/ARandom06_Br06CoclustAr06p1-p1-d10/ARandom06_Br06CoclustAr06p1-p1-d10.pkl
N particles initial (before exclusion):
    subclass_name  ARandom06  Br06CoclustAr06p1-p1-d10
    tomo_id                                           
    alpha                300                       250
    Total                300                       250
Pickled  MPS object to coloc/after_exclusion_particles/ARandom06_Br06CoclustAr06p1-p1-d10/ARandom06_Br06CoclustAr06p1-p1-d10.pkl
N particles after exclusion:
    subclass_name  ARandom06  Br06CoclustAr06p1-p1-

In [119]:
# colocalization name
coloc_name = 'Br06CoclustAr06p1-p1-d10_ARandom06'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: Br06CoclustAr06p1-p1-d10_ARandom06
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/Br06CoclustAr06p1-p1-d10_ARandom06/Br06CoclustAr06p1-p1-d10_ARandom06.pkl
N particles initial (before exclusion):
    subclass_name  ARandom06  Br06CoclustAr06p1-p1-d10
    tomo_id                                           
    alpha                300                       250
    Total                300                       250
Pickled  MPS object to coloc/after_exclusion_particles/Br06CoclustAr06p1-p1-d10_ARandom06/Br06CoclustAr06p1-p1-d10_ARandom06.pkl
N particles after exclusion:
    subclass_name  ARandom06  Br06CoclustAr06p1-p1-

In [120]:
# colocalization name
coloc_name = 'ARandom06_Br06CoclustAr06p07-p07-d10'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: ARandom06_Br06CoclustAr06p07-p07-d10
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/ARandom06_Br06CoclustAr06p07-p07-d10/ARandom06_Br06CoclustAr06p07-p07-d10.pkl
N particles initial (before exclusion):
    subclass_name  ARandom06  Br06CoclustAr06p07-p07-d10
    tomo_id                                             
    alpha                300                         250
    Total                300                         250
Pickled  MPS object to coloc/after_exclusion_particles/ARandom06_Br06CoclustAr06p07-p07-d10/ARandom06_Br06CoclustAr06p07-p07-d10.pkl
N particles after exclusion:
    subclass_name  ARandom06  Br0

In [121]:
# colocalization name
coloc_name = 'Br06CoclustAr06p07-p07-d10_ARandom06'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: Br06CoclustAr06p07-p07-d10_ARandom06
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/Br06CoclustAr06p07-p07-d10_ARandom06/Br06CoclustAr06p07-p07-d10_ARandom06.pkl
N particles initial (before exclusion):
    subclass_name  ARandom06  Br06CoclustAr06p07-p07-d10
    tomo_id                                             
    alpha                300                         250
    Total                300                         250
Pickled  MPS object to coloc/after_exclusion_particles/Br06CoclustAr06p07-p07-d10_ARandom06/Br06CoclustAr06p07-p07-d10_ARandom06.pkl
N particles after exclusion:
    subclass_name  ARandom06  Br0

In [122]:
# colocalization name
coloc_name = 'ARandom06_Br06CoclustAr06p03-p03-d10'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: ARandom06_Br06CoclustAr06p03-p03-d10
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/ARandom06_Br06CoclustAr06p03-p03-d10/ARandom06_Br06CoclustAr06p03-p03-d10.pkl
N particles initial (before exclusion):
    subclass_name  ARandom06  Br06CoclustAr06p03-p03-d10
    tomo_id                                             
    alpha                300                         250
    Total                300                         250
Pickled  MPS object to coloc/after_exclusion_particles/ARandom06_Br06CoclustAr06p03-p03-d10/ARandom06_Br06CoclustAr06p03-p03-d10.pkl
N particles after exclusion:
    subclass_name  ARandom06  Br0

In [123]:
# colocalization name
coloc_name = 'Br06CoclustAr06p03-p03-d10_ARandom06'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: Br06CoclustAr06p03-p03-d10_ARandom06
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/Br06CoclustAr06p03-p03-d10_ARandom06/Br06CoclustAr06p03-p03-d10_ARandom06.pkl
N particles initial (before exclusion):
    subclass_name  ARandom06  Br06CoclustAr06p03-p03-d10
    tomo_id                                             
    alpha                300                         250
    Total                300                         250
Pickled  MPS object to coloc/after_exclusion_particles/Br06CoclustAr06p03-p03-d10_ARandom06/Br06CoclustAr06p03-p03-d10_ARandom06.pkl
N particles after exclusion:
    subclass_name  ARandom06  Br0

#### Size 0.6, full overlap, 1-to-1

In [124]:
# colocalization name
coloc_name = 'ARandom06_Br06CoclustAr06p03-pkd-d10'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: ARandom06_Br06CoclustAr06p03-pkd-d10
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/ARandom06_Br06CoclustAr06p03-pkd-d10/ARandom06_Br06CoclustAr06p03-pkd-d10.pkl
N particles initial (before exclusion):
    subclass_name  ARandom06  Br06CoclustAr06p03-pkd-d10
    tomo_id                                             
    alpha                300                         250
    Total                300                         250
Pickled  MPS object to coloc/after_exclusion_particles/ARandom06_Br06CoclustAr06p03-pkd-d10/ARandom06_Br06CoclustAr06p03-pkd-d10.pkl
N particles after exclusion:
    subclass_name  ARandom06  Br0

In [125]:
# colocalization name
coloc_name = 'Br06CoclustAr06p03-pkd-d10_ARandom06'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: Br06CoclustAr06p03-pkd-d10_ARandom06
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/Br06CoclustAr06p03-pkd-d10_ARandom06/Br06CoclustAr06p03-pkd-d10_ARandom06.pkl
N particles initial (before exclusion):
    subclass_name  ARandom06  Br06CoclustAr06p03-pkd-d10
    tomo_id                                             
    alpha                300                         250
    Total                300                         250
Pickled  MPS object to coloc/after_exclusion_particles/Br06CoclustAr06p03-pkd-d10_ARandom06/Br06CoclustAr06p03-pkd-d10_ARandom06.pkl
N particles after exclusion:
    subclass_name  ARandom06  Br0

In [126]:
# colocalization name
coloc_name = 'ARandom06_Br06CoclustAr06p02-pkd-d10'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: ARandom06_Br06CoclustAr06p02-pkd-d10
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/ARandom06_Br06CoclustAr06p02-pkd-d10/ARandom06_Br06CoclustAr06p02-pkd-d10.pkl
N particles initial (before exclusion):
    subclass_name  ARandom06  Br06CoclustAr06p02-pkd-d10
    tomo_id                                             
    alpha                300                         250
    Total                300                         250
Pickled  MPS object to coloc/after_exclusion_particles/ARandom06_Br06CoclustAr06p02-pkd-d10/ARandom06_Br06CoclustAr06p02-pkd-d10.pkl
N particles after exclusion:
    subclass_name  ARandom06  Br0

In [127]:
# colocalization name
coloc_name = 'Br06CoclustAr06p02-pkd-d10_ARandom06'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: Br06CoclustAr06p02-pkd-d10_ARandom06
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/Br06CoclustAr06p02-pkd-d10_ARandom06/Br06CoclustAr06p02-pkd-d10_ARandom06.pkl
N particles initial (before exclusion):
    subclass_name  ARandom06  Br06CoclustAr06p02-pkd-d10
    tomo_id                                             
    alpha                300                         250
    Total                300                         250
Pickled  MPS object to coloc/after_exclusion_particles/Br06CoclustAr06p02-pkd-d10_ARandom06/Br06CoclustAr06p02-pkd-d10_ARandom06.pkl
N particles after exclusion:
    subclass_name  ARandom06  Br0

In [128]:
# colocalization name
coloc_name = 'ARandom06_Br06CoclustAr06p015-pkd-d10'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: ARandom06_Br06CoclustAr06p015-pkd-d10
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/ARandom06_Br06CoclustAr06p015-pkd-d10/ARandom06_Br06CoclustAr06p015-pkd-d10.pkl
N particles initial (before exclusion):
    subclass_name  ARandom06  Br06CoclustAr06p015-pkd-d10
    tomo_id                                              
    alpha                300                          250
    Total                300                          250
Pickled  MPS object to coloc/after_exclusion_particles/ARandom06_Br06CoclustAr06p015-pkd-d10/ARandom06_Br06CoclustAr06p015-pkd-d10.pkl
N particles after exclusion:
    subclass_name  ARand

In [129]:
# colocalization name
coloc_name = 'Br06CoclustAr06p015-pkd-d10_ARandom06'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: Br06CoclustAr06p015-pkd-d10_ARandom06
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/Br06CoclustAr06p015-pkd-d10_ARandom06/Br06CoclustAr06p015-pkd-d10_ARandom06.pkl
N particles initial (before exclusion):
    subclass_name  ARandom06  Br06CoclustAr06p015-pkd-d10
    tomo_id                                              
    alpha                300                          250
    Total                300                          250
Pickled  MPS object to coloc/after_exclusion_particles/Br06CoclustAr06p015-pkd-d10_ARandom06/Br06CoclustAr06p015-pkd-d10_ARandom06.pkl
N particles after exclusion:
    subclass_name  ARand

### Interaction mode 3 A-C

#### Size full and 08

In [130]:
# colocalization name
coloc_name = 'ARandomFull_CrfCoclustArfp1-p1-d10'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: ARandomFull_CrfCoclustArfp1-p1-d10
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/ARandomFull_CrfCoclustArfp1-p1-d10/ARandomFull_CrfCoclustArfp1-p1-d10.pkl
N particles initial (before exclusion):
    subclass_name  ARandomFull  CrfCoclustArfp1-p1-d10
    tomo_id                                           
    alpha                  300                      50
    Total                  300                      50
Pickled  MPS object to coloc/after_exclusion_particles/ARandomFull_CrfCoclustArfp1-p1-d10/ARandomFull_CrfCoclustArfp1-p1-d10.pkl
N particles after exclusion:
    subclass_name  ARandomFull  CrfCoclustArfp1-p1-

In [131]:
# colocalization name
coloc_name = 'CrfCoclustArfp1-p1-d10_ARandomFull'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: CrfCoclustArfp1-p1-d10_ARandomFull
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/CrfCoclustArfp1-p1-d10_ARandomFull/CrfCoclustArfp1-p1-d10_ARandomFull.pkl
N particles initial (before exclusion):
    subclass_name  ARandomFull  CrfCoclustArfp1-p1-d10
    tomo_id                                           
    alpha                  300                      50
    Total                  300                      50
Pickled  MPS object to coloc/after_exclusion_particles/CrfCoclustArfp1-p1-d10_ARandomFull/CrfCoclustArfp1-p1-d10_ARandomFull.pkl
N particles after exclusion:
    subclass_name  ARandomFull  CrfCoclustArfp1-p1-

In [132]:
# colocalization name
coloc_name = 'ARandomFull_CrfCoclustArfp07-p07-d10'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: ARandomFull_CrfCoclustArfp07-p07-d10
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/ARandomFull_CrfCoclustArfp07-p07-d10/ARandomFull_CrfCoclustArfp07-p07-d10.pkl
N particles initial (before exclusion):
    subclass_name  ARandomFull  CrfCoclustArfp07-p07-d10
    tomo_id                                             
    alpha                  300                        50
    Total                  300                        50
Pickled  MPS object to coloc/after_exclusion_particles/ARandomFull_CrfCoclustArfp07-p07-d10/ARandomFull_CrfCoclustArfp07-p07-d10.pkl
N particles after exclusion:
    subclass_name  ARandomFull  C

In [133]:
# colocalization name
coloc_name = 'CrfCoclustArfp07-p07-d10_ARandomFull'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: CrfCoclustArfp07-p07-d10_ARandomFull
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/CrfCoclustArfp07-p07-d10_ARandomFull/CrfCoclustArfp07-p07-d10_ARandomFull.pkl
N particles initial (before exclusion):
    subclass_name  ARandomFull  CrfCoclustArfp07-p07-d10
    tomo_id                                             
    alpha                  300                        50
    Total                  300                        50
Pickled  MPS object to coloc/after_exclusion_particles/CrfCoclustArfp07-p07-d10_ARandomFull/CrfCoclustArfp07-p07-d10_ARandomFull.pkl
N particles after exclusion:
    subclass_name  ARandomFull  C

In [134]:
# colocalization name
coloc_name = 'ARandomFull_CrfCoclustArfp06-p06-d10'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: ARandomFull_CrfCoclustArfp06-p06-d10
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/ARandomFull_CrfCoclustArfp06-p06-d10/ARandomFull_CrfCoclustArfp06-p06-d10.pkl
N particles initial (before exclusion):
    subclass_name  ARandomFull  CrfCoclustArfp06-p06-d10
    tomo_id                                             
    alpha                  300                        50
    Total                  300                        50
Pickled  MPS object to coloc/after_exclusion_particles/ARandomFull_CrfCoclustArfp06-p06-d10/ARandomFull_CrfCoclustArfp06-p06-d10.pkl
N particles after exclusion:
    subclass_name  ARandomFull  C

In [135]:
# colocalization name
coloc_name = 'CrfCoclustArfp06-p06-d10_ARandomFull'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: CrfCoclustArfp06-p06-d10_ARandomFull
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/CrfCoclustArfp06-p06-d10_ARandomFull/CrfCoclustArfp06-p06-d10_ARandomFull.pkl
N particles initial (before exclusion):
    subclass_name  ARandomFull  CrfCoclustArfp06-p06-d10
    tomo_id                                             
    alpha                  300                        50
    Total                  300                        50
Pickled  MPS object to coloc/after_exclusion_particles/CrfCoclustArfp06-p06-d10_ARandomFull/CrfCoclustArfp06-p06-d10_ARandomFull.pkl
N particles after exclusion:
    subclass_name  ARandomFull  C

In [136]:
# colocalization name
coloc_name = 'ARandomFull_CrfCoclustArfp05-p05-d10'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: ARandomFull_CrfCoclustArfp05-p05-d10
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/ARandomFull_CrfCoclustArfp05-p05-d10/ARandomFull_CrfCoclustArfp05-p05-d10.pkl
N particles initial (before exclusion):
    subclass_name  ARandomFull  CrfCoclustArfp05-p05-d10
    tomo_id                                             
    alpha                  300                        50
    Total                  300                        50
Pickled  MPS object to coloc/after_exclusion_particles/ARandomFull_CrfCoclustArfp05-p05-d10/ARandomFull_CrfCoclustArfp05-p05-d10.pkl
N particles after exclusion:
    subclass_name  ARandomFull  C

In [137]:
# colocalization name
coloc_name = 'CrfCoclustArfp05-p05-d10_ARandomFull'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: CrfCoclustArfp05-p05-d10_ARandomFull
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/CrfCoclustArfp05-p05-d10_ARandomFull/CrfCoclustArfp05-p05-d10_ARandomFull.pkl
N particles initial (before exclusion):
    subclass_name  ARandomFull  CrfCoclustArfp05-p05-d10
    tomo_id                                             
    alpha                  300                        50
    Total                  300                        50
Pickled  MPS object to coloc/after_exclusion_particles/CrfCoclustArfp05-p05-d10_ARandomFull/CrfCoclustArfp05-p05-d10_ARandomFull.pkl
N particles after exclusion:
    subclass_name  ARandomFull  C

In [138]:
# colocalization name
coloc_name = 'ARandomFull_CrfCoclustArfp04-p04-d10'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: ARandomFull_CrfCoclustArfp04-p04-d10
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/ARandomFull_CrfCoclustArfp04-p04-d10/ARandomFull_CrfCoclustArfp04-p04-d10.pkl
N particles initial (before exclusion):
    subclass_name  ARandomFull  CrfCoclustArfp04-p04-d10
    tomo_id                                             
    alpha                  300                        50
    Total                  300                        50
Pickled  MPS object to coloc/after_exclusion_particles/ARandomFull_CrfCoclustArfp04-p04-d10/ARandomFull_CrfCoclustArfp04-p04-d10.pkl
N particles after exclusion:
    subclass_name  ARandomFull  C

In [139]:
# colocalization name
coloc_name = 'CrfCoclustArfp04-p04-d10_ARandomFull'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: CrfCoclustArfp04-p04-d10_ARandomFull
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/CrfCoclustArfp04-p04-d10_ARandomFull/CrfCoclustArfp04-p04-d10_ARandomFull.pkl
N particles initial (before exclusion):
    subclass_name  ARandomFull  CrfCoclustArfp04-p04-d10
    tomo_id                                             
    alpha                  300                        50
    Total                  300                        50
Pickled  MPS object to coloc/after_exclusion_particles/CrfCoclustArfp04-p04-d10_ARandomFull/CrfCoclustArfp04-p04-d10_ARandomFull.pkl
N particles after exclusion:
    subclass_name  ARandomFull  C

In [140]:
# colocalization name
coloc_name = 'ARandomFull_CrfCoclustArfp03-p03-d10'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: ARandomFull_CrfCoclustArfp03-p03-d10
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/ARandomFull_CrfCoclustArfp03-p03-d10/ARandomFull_CrfCoclustArfp03-p03-d10.pkl
N particles initial (before exclusion):
    subclass_name  ARandomFull  CrfCoclustArfp03-p03-d10
    tomo_id                                             
    alpha                  300                        50
    Total                  300                        50
Pickled  MPS object to coloc/after_exclusion_particles/ARandomFull_CrfCoclustArfp03-p03-d10/ARandomFull_CrfCoclustArfp03-p03-d10.pkl
N particles after exclusion:
    subclass_name  ARandomFull  C

In [141]:
# colocalization name
coloc_name = 'CrfCoclustArfp03-p03-d10_ARandomFull'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: CrfCoclustArfp03-p03-d10_ARandomFull
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/CrfCoclustArfp03-p03-d10_ARandomFull/CrfCoclustArfp03-p03-d10_ARandomFull.pkl
N particles initial (before exclusion):
    subclass_name  ARandomFull  CrfCoclustArfp03-p03-d10
    tomo_id                                             
    alpha                  300                        50
    Total                  300                        50
Pickled  MPS object to coloc/after_exclusion_particles/CrfCoclustArfp03-p03-d10_ARandomFull/CrfCoclustArfp03-p03-d10_ARandomFull.pkl
N particles after exclusion:
    subclass_name  ARandomFull  C

#### Size full 1-to-1

In [142]:
# colocalization name
coloc_name = 'ARandomFull_CrfCoclustArfp015-pkd-d10'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: ARandomFull_CrfCoclustArfp015-pkd-d10
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/ARandomFull_CrfCoclustArfp015-pkd-d10/ARandomFull_CrfCoclustArfp015-pkd-d10.pkl
N particles initial (before exclusion):
    subclass_name  ARandomFull  CrfCoclustArfp015-pkd-d10
    tomo_id                                              
    alpha                  300                         50
    Total                  300                         50
Pickled  MPS object to coloc/after_exclusion_particles/ARandomFull_CrfCoclustArfp015-pkd-d10/ARandomFull_CrfCoclustArfp015-pkd-d10.pkl
N particles after exclusion:
    subclass_name  ARand

In [143]:
# colocalization name
coloc_name = 'CrfCoclustArfp015-pkd-d10_ARandomFull'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: CrfCoclustArfp015-pkd-d10_ARandomFull
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/CrfCoclustArfp015-pkd-d10_ARandomFull/CrfCoclustArfp015-pkd-d10_ARandomFull.pkl
N particles initial (before exclusion):
    subclass_name  ARandomFull  CrfCoclustArfp015-pkd-d10
    tomo_id                                              
    alpha                  300                         50
    Total                  300                         50
Pickled  MPS object to coloc/after_exclusion_particles/CrfCoclustArfp015-pkd-d10_ARandomFull/CrfCoclustArfp015-pkd-d10_ARandomFull.pkl
N particles after exclusion:
    subclass_name  ARand

In [144]:
# colocalization name
coloc_name = 'ARandomFull_CrfCoclustArfp01-pkd-d10'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: ARandomFull_CrfCoclustArfp01-pkd-d10
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/ARandomFull_CrfCoclustArfp01-pkd-d10/ARandomFull_CrfCoclustArfp01-pkd-d10.pkl
N particles initial (before exclusion):
    subclass_name  ARandomFull  CrfCoclustArfp01-pkd-d10
    tomo_id                                             
    alpha                  300                        50
    Total                  300                        50
Pickled  MPS object to coloc/after_exclusion_particles/ARandomFull_CrfCoclustArfp01-pkd-d10/ARandomFull_CrfCoclustArfp01-pkd-d10.pkl
N particles after exclusion:
    subclass_name  ARandomFull  C

In [145]:
# colocalization name
coloc_name = 'CrfCoclustArfp01-pkd-d10_ARandomFull'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: CrfCoclustArfp01-pkd-d10_ARandomFull
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/CrfCoclustArfp01-pkd-d10_ARandomFull/CrfCoclustArfp01-pkd-d10_ARandomFull.pkl
N particles initial (before exclusion):
    subclass_name  ARandomFull  CrfCoclustArfp01-pkd-d10
    tomo_id                                             
    alpha                  300                        50
    Total                  300                        50
Pickled  MPS object to coloc/after_exclusion_particles/CrfCoclustArfp01-pkd-d10_ARandomFull/CrfCoclustArfp01-pkd-d10_ARandomFull.pkl
N particles after exclusion:
    subclass_name  ARandomFull  C

In [146]:
# colocalization name
coloc_name = 'ARandomFull_CrfCoclustArfp005-pkd-d10'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: ARandomFull_CrfCoclustArfp005-pkd-d10
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/ARandomFull_CrfCoclustArfp005-pkd-d10/ARandomFull_CrfCoclustArfp005-pkd-d10.pkl
N particles initial (before exclusion):
    subclass_name  ARandomFull  CrfCoclustArfp005-pkd-d10
    tomo_id                                              
    alpha                  300                         50
    Total                  300                         50
Pickled  MPS object to coloc/after_exclusion_particles/ARandomFull_CrfCoclustArfp005-pkd-d10/ARandomFull_CrfCoclustArfp005-pkd-d10.pkl
N particles after exclusion:
    subclass_name  ARand

In [147]:
# colocalization name
coloc_name = 'CrfCoclustArfp005-pkd-d10_ARandomFull'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: CrfCoclustArfp005-pkd-d10_ARandomFull
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/CrfCoclustArfp005-pkd-d10_ARandomFull/CrfCoclustArfp005-pkd-d10_ARandomFull.pkl
N particles initial (before exclusion):
    subclass_name  ARandomFull  CrfCoclustArfp005-pkd-d10
    tomo_id                                              
    alpha                  300                         50
    Total                  300                         50
Pickled  MPS object to coloc/after_exclusion_particles/CrfCoclustArfp005-pkd-d10_ARandomFull/CrfCoclustArfp005-pkd-d10_ARandomFull.pkl
N particles after exclusion:
    subclass_name  ARand

#### Size 06 1-to-1

In [148]:
# colocalization name
coloc_name = 'ARandom06_Cr06CoclustAr06p015-pkd-d10'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: ARandom06_Cr06CoclustAr06p015-pkd-d10
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/ARandom06_Cr06CoclustAr06p015-pkd-d10/ARandom06_Cr06CoclustAr06p015-pkd-d10.pkl
N particles initial (before exclusion):
    subclass_name  ARandom06  Cr06CoclustAr06p015-pkd-d10
    tomo_id                                              
    alpha                300                           50
    Total                300                           50
Pickled  MPS object to coloc/after_exclusion_particles/ARandom06_Cr06CoclustAr06p015-pkd-d10/ARandom06_Cr06CoclustAr06p015-pkd-d10.pkl
N particles after exclusion:
    subclass_name  ARand

In [149]:
# colocalization name
coloc_name = 'Cr06CoclustAr06p015-pkd-d10_ARandom06'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: Cr06CoclustAr06p015-pkd-d10_ARandom06
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/Cr06CoclustAr06p015-pkd-d10_ARandom06/Cr06CoclustAr06p015-pkd-d10_ARandom06.pkl
N particles initial (before exclusion):
    subclass_name  ARandom06  Cr06CoclustAr06p015-pkd-d10
    tomo_id                                              
    alpha                300                           50
    Total                300                           50
Pickled  MPS object to coloc/after_exclusion_particles/Cr06CoclustAr06p015-pkd-d10_ARandom06/Cr06CoclustAr06p015-pkd-d10_ARandom06.pkl
N particles after exclusion:
    subclass_name  ARand

In [150]:
# colocalization name
coloc_name = 'ARandom06_Cr06CoclustAr06p01-pkd-d10'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: ARandom06_Cr06CoclustAr06p01-pkd-d10
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/ARandom06_Cr06CoclustAr06p01-pkd-d10/ARandom06_Cr06CoclustAr06p01-pkd-d10.pkl
N particles initial (before exclusion):
    subclass_name  ARandom06  Cr06CoclustAr06p01-pkd-d10
    tomo_id                                             
    alpha                300                          50
    Total                300                          50
Pickled  MPS object to coloc/after_exclusion_particles/ARandom06_Cr06CoclustAr06p01-pkd-d10/ARandom06_Cr06CoclustAr06p01-pkd-d10.pkl
N particles after exclusion:
    subclass_name  ARandom06  Cr0

In [151]:
# colocalization name
coloc_name = 'Cr06CoclustAr06p01-pkd-d10_ARandom06'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: Cr06CoclustAr06p01-pkd-d10_ARandom06
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/Cr06CoclustAr06p01-pkd-d10_ARandom06/Cr06CoclustAr06p01-pkd-d10_ARandom06.pkl
N particles initial (before exclusion):
    subclass_name  ARandom06  Cr06CoclustAr06p01-pkd-d10
    tomo_id                                             
    alpha                300                          50
    Total                300                          50
Pickled  MPS object to coloc/after_exclusion_particles/Cr06CoclustAr06p01-pkd-d10_ARandom06/Cr06CoclustAr06p01-pkd-d10_ARandom06.pkl
N particles after exclusion:
    subclass_name  ARandom06  Cr0

In [152]:
# colocalization name
coloc_name = 'ARandom06_Cr06CoclustAr06p005-pkd-d10'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: ARandom06_Cr06CoclustAr06p005-pkd-d10
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/ARandom06_Cr06CoclustAr06p005-pkd-d10/ARandom06_Cr06CoclustAr06p005-pkd-d10.pkl
N particles initial (before exclusion):
    subclass_name  ARandom06  Cr06CoclustAr06p005-pkd-d10
    tomo_id                                              
    alpha                300                           50
    Total                300                           50
Pickled  MPS object to coloc/after_exclusion_particles/ARandom06_Cr06CoclustAr06p005-pkd-d10/ARandom06_Cr06CoclustAr06p005-pkd-d10.pkl
N particles after exclusion:
    subclass_name  ARand

In [153]:
# colocalization name
coloc_name = 'Cr06CoclustAr06p005-pkd-d10_ARandom06'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: Cr06CoclustAr06p005-pkd-d10_ARandom06
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/Cr06CoclustAr06p005-pkd-d10_ARandom06/Cr06CoclustAr06p005-pkd-d10_ARandom06.pkl
N particles initial (before exclusion):
    subclass_name  ARandom06  Cr06CoclustAr06p005-pkd-d10
    tomo_id                                              
    alpha                300                           50
    Total                300                           50
Pickled  MPS object to coloc/after_exclusion_particles/Cr06CoclustAr06p005-pkd-d10_ARandom06/Cr06CoclustAr06p005-pkd-d10_ARandom06.pkl
N particles after exclusion:
    subclass_name  ARand

## 3-colocalizations

### Random full

In [154]:
# Colocalization

# colocalization name
coloc_name = 'ARandomFull_BRandomFull_CRandomFull'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization cases: ARandomFull_BRandomFull_CRandomFull, ARandomFull_BRandomFull, ARandomFull_CRandomFull
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/ARandomFull_BRandomFull_CRandomFull/ARandomFull_BRandomFull_CRandomFull.pkl
N particles initial (before exclusion):
    subclass_name  ARandomFull  BRandomFull  CRandomFull
    tomo_id                                             
    alpha                  300          250           50
    Total                  300          250           50
Pickled  MPS object to coloc/after_exclusion_particles/ARandomFull_BRandomFull_CRandomFull/ARandomFull_BRandomFull_CRandomFull.pkl
N particles afte

In [155]:
# Colocalization

# colocalization name
coloc_name = 'CRandomFull_ARandomFull_BRandomFull'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc')
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization cases: CRandomFull_ARandomFull_BRandomFull, CRandomFull_ARandomFull, CRandomFull_BRandomFull
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/CRandomFull_ARandomFull_BRandomFull/CRandomFull_ARandomFull_BRandomFull.pkl
N particles initial (before exclusion):
    subclass_name  ARandomFull  BRandomFull  CRandomFull
    tomo_id                                             
    alpha                  300          250           50
    Total                  300          250           50
Pickled  MPS object to coloc/after_exclusion_particles/CRandomFull_ARandomFull_BRandomFull/CRandomFull_ARandomFull_BRandomFull.pkl
N particles afte

### Interaction mode 3

#### Full, 1-to-1, independent interacting subsets

In [156]:
# Colocalization

# colocalization name
coloc_name = 'ARandomFull_BrfCoclustArfp015-pkd-d10_CrfCoclustArfp015-pkd-d10'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc', full_coloc=False)
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: ARandomFull_BrfCoclustArfp015-pkd-d10_CrfCoclustArfp015-pkd-d10
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/ARandomFull_BrfCoclustArfp015-pkd-d10_CrfCoclustArfp015-pkd-d10/ARandomFull_BrfCoclustArfp015-pkd-d10_CrfCoclustArfp015-pkd-d10.pkl
N particles initial (before exclusion):
    subclass_name  ARandomFull  BrfCoclustArfp015-pkd-d10  CrfCoclustArfp015-pkd-d10
    tomo_id                                                                         
    alpha                  300                        250                         50
    Total                  300                        250                         50
Pi

In [157]:
# Colocalization

# colocalization name
coloc_name = 'CrfCoclustArfp015-pkd-d10_ARandomFull_BrfCoclustArfp015-pkd-d10'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc', full_coloc=False)
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: CrfCoclustArfp015-pkd-d10_ARandomFull_BrfCoclustArfp015-pkd-d10
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/CrfCoclustArfp015-pkd-d10_ARandomFull_BrfCoclustArfp015-pkd-d10/CrfCoclustArfp015-pkd-d10_ARandomFull_BrfCoclustArfp015-pkd-d10.pkl
N particles initial (before exclusion):
    subclass_name  ARandomFull  BrfCoclustArfp015-pkd-d10  CrfCoclustArfp015-pkd-d10
    tomo_id                                                                         
    alpha                  300                        250                         50
    Total                  300                        250                         50
Pi

In [158]:
# Colocalization

# colocalization name
coloc_name = 'ARandomFull_BrfCoclustArfp01-pkd-d10_CrfCoclustArfp01-pkd-d10'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc', full_coloc=False)
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: ARandomFull_BrfCoclustArfp01-pkd-d10_CrfCoclustArfp01-pkd-d10
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/ARandomFull_BrfCoclustArfp01-pkd-d10_CrfCoclustArfp01-pkd-d10/ARandomFull_BrfCoclustArfp01-pkd-d10_CrfCoclustArfp01-pkd-d10.pkl
N particles initial (before exclusion):
    subclass_name  ARandomFull  BrfCoclustArfp01-pkd-d10  CrfCoclustArfp01-pkd-d10
    tomo_id                                                                       
    alpha                  300                       250                        50
    Total                  300                       250                        50
Pickled  MPS obj

In [159]:
# Colocalization

# colocalization name
coloc_name = 'CrfCoclustArfp01-pkd-d10_ARandomFull_BrfCoclustArfp01-pkd-d10'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc', full_coloc=False)
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: CrfCoclustArfp01-pkd-d10_ARandomFull_BrfCoclustArfp01-pkd-d10
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/CrfCoclustArfp01-pkd-d10_ARandomFull_BrfCoclustArfp01-pkd-d10/CrfCoclustArfp01-pkd-d10_ARandomFull_BrfCoclustArfp01-pkd-d10.pkl
N particles initial (before exclusion):
    subclass_name  ARandomFull  BrfCoclustArfp01-pkd-d10  CrfCoclustArfp01-pkd-d10
    tomo_id                                                                       
    alpha                  300                       250                        50
    Total                  300                       250                        50
Pickled  MPS obj

In [160]:
# Colocalization

# colocalization name
coloc_name = 'ARandomFull_BrfCoclustArfp005-pkd-d10_CrfCoclustArfp005-pkd-d10'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc', full_coloc=False)
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: ARandomFull_BrfCoclustArfp005-pkd-d10_CrfCoclustArfp005-pkd-d10
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/ARandomFull_BrfCoclustArfp005-pkd-d10_CrfCoclustArfp005-pkd-d10/ARandomFull_BrfCoclustArfp005-pkd-d10_CrfCoclustArfp005-pkd-d10.pkl
N particles initial (before exclusion):
    subclass_name  ARandomFull  BrfCoclustArfp005-pkd-d10  CrfCoclustArfp005-pkd-d10
    tomo_id                                                                         
    alpha                  300                        250                         50
    Total                  300                        250                         50
Pi

In [161]:
# Colocalization

# colocalization name
coloc_name = 'CrfCoclustArfp005-pkd-d10_ARandomFull_BrfCoclustArfp005-pkd-d10'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc', full_coloc=False)
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: CrfCoclustArfp005-pkd-d10_ARandomFull_BrfCoclustArfp005-pkd-d10
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/CrfCoclustArfp005-pkd-d10_ARandomFull_BrfCoclustArfp005-pkd-d10/CrfCoclustArfp005-pkd-d10_ARandomFull_BrfCoclustArfp005-pkd-d10.pkl
N particles initial (before exclusion):
    subclass_name  ARandomFull  BrfCoclustArfp005-pkd-d10  CrfCoclustArfp005-pkd-d10
    tomo_id                                                                         
    alpha                  300                        250                         50
    Total                  300                        250                         50
Pi

#### Full, 1-to-1, common interacting subset

In [162]:
# Colocalization

# colocalization name
coloc_name = 'ARandomFull_BrfCoclustArfp015k-pkd-d10_CrfCoclustArfp015k-pkd-d10'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc', full_coloc=False)
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: ARandomFull_BrfCoclustArfp015k-pkd-d10_CrfCoclustArfp015k-pkd-d10
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/ARandomFull_BrfCoclustArfp015k-pkd-d10_CrfCoclustArfp015k-pkd-d10/ARandomFull_BrfCoclustArfp015k-pkd-d10_CrfCoclustArfp015k-pkd-d10.pkl
N particles initial (before exclusion):
    subclass_name  ARandomFull  BrfCoclustArfp015k-pkd-d10  CrfCoclustArfp015k-pkd-d10
    tomo_id                                                                           
    alpha                  300                         250                          50
    Total                  300                         250                 

In [163]:
# Colocalization

# colocalization name
coloc_name = 'CrfCoclustArfp015k-pkd-d10_ARandomFull_BrfCoclustArfp015k-pkd-d10'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc', full_coloc=False)
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: CrfCoclustArfp015k-pkd-d10_ARandomFull_BrfCoclustArfp015k-pkd-d10
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/CrfCoclustArfp015k-pkd-d10_ARandomFull_BrfCoclustArfp015k-pkd-d10/CrfCoclustArfp015k-pkd-d10_ARandomFull_BrfCoclustArfp015k-pkd-d10.pkl
N particles initial (before exclusion):
    subclass_name  ARandomFull  BrfCoclustArfp015k-pkd-d10  CrfCoclustArfp015k-pkd-d10
    tomo_id                                                                           
    alpha                  300                         250                          50
    Total                  300                         250                 

In [164]:
# Colocalization

# colocalization name
coloc_name = 'ARandomFull_BrfCoclustArfp01k-pkd-d10_CrfCoclustArfp01k-pkd-d10'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc', full_coloc=False)
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: ARandomFull_BrfCoclustArfp01k-pkd-d10_CrfCoclustArfp01k-pkd-d10
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/ARandomFull_BrfCoclustArfp01k-pkd-d10_CrfCoclustArfp01k-pkd-d10/ARandomFull_BrfCoclustArfp01k-pkd-d10_CrfCoclustArfp01k-pkd-d10.pkl
N particles initial (before exclusion):
    subclass_name  ARandomFull  BrfCoclustArfp01k-pkd-d10  CrfCoclustArfp01k-pkd-d10
    tomo_id                                                                         
    alpha                  300                        250                         50
    Total                  300                        250                         50
Pi

In [165]:
# Colocalization

# colocalization name
coloc_name = 'CrfCoclustArfp01k-pkd-d10_ARandomFull_BrfCoclustArfp01k-pkd-d10'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc', full_coloc=False)
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: CrfCoclustArfp01k-pkd-d10_ARandomFull_BrfCoclustArfp01k-pkd-d10
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/CrfCoclustArfp01k-pkd-d10_ARandomFull_BrfCoclustArfp01k-pkd-d10/CrfCoclustArfp01k-pkd-d10_ARandomFull_BrfCoclustArfp01k-pkd-d10.pkl
N particles initial (before exclusion):
    subclass_name  ARandomFull  BrfCoclustArfp01k-pkd-d10  CrfCoclustArfp01k-pkd-d10
    tomo_id                                                                         
    alpha                  300                        250                         50
    Total                  300                        250                         50
Pi

In [166]:
# Colocalization

# colocalization name
coloc_name = 'ARandomFull_BrfCoclustArfp005k-pkd-d10_CrfCoclustArfp005k-pkd-d10'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc', full_coloc=False)
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: ARandomFull_BrfCoclustArfp005k-pkd-d10_CrfCoclustArfp005k-pkd-d10
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/ARandomFull_BrfCoclustArfp005k-pkd-d10_CrfCoclustArfp005k-pkd-d10/ARandomFull_BrfCoclustArfp005k-pkd-d10_CrfCoclustArfp005k-pkd-d10.pkl
N particles initial (before exclusion):
    subclass_name  ARandomFull  BrfCoclustArfp005k-pkd-d10  CrfCoclustArfp005k-pkd-d10
    tomo_id                                                                           
    alpha                  300                         250                          50
    Total                  300                         250                 

In [167]:
# Colocalization

# colocalization name
coloc_name = 'CrfCoclustArfp005k-pkd-d10_ARandomFull_BrfCoclustArfp005k-pkd-d10'

# select particles 
set_names = col_func.get_names(coloc_name, mode=name_mode)
mps_local = mps.select(class_names=set_names, update=False)

# colocalize
col = ColocLite(dir_='coloc', full_coloc=False)
col.colocalize(
    coloc_case=coloc_name, particles=mps_local, distance=distance,
    n_simul=n_simul)

Colocalization parameters:
    Colocalization name: CrfCoclustArfp005k-pkd-d10_ARandomFull_BrfCoclustArfp005k-pkd-d10
    Colocalization distance [nm]: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 
    Calculate columns: True, columns distance factor 2
    Exclusion distance [nm]: None
    Make convex hull: False

Tomo pixel size [nm]
    alpha: 1
    bravo: 1
    charlie: 1
Pickled  MPS object to coloc/input_particles/CrfCoclustArfp005k-pkd-d10_ARandomFull_BrfCoclustArfp005k-pkd-d10/CrfCoclustArfp005k-pkd-d10_ARandomFull_BrfCoclustArfp005k-pkd-d10.pkl
N particles initial (before exclusion):
    subclass_name  ARandomFull  BrfCoclustArfp005k-pkd-d10  CrfCoclustArfp005k-pkd-d10
    tomo_id                                                                           
    alpha                  300                         250                          50
    Total                  300                         250                 